<!-- Badges: -->
[![Donate](https://img.shields.io/badge/Donate-PayPal-green.svg?logo=paypal&style=flat-square)](https://www.paypal.me/CamponogaraViera/100)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuCAI-Lab/ibm2021-open-science-prize/blob/dev/simulation.ipynb)
[![License](https://img.shields.io/github/license/QuCAI-Lab/ibm2021-open-science-prize.svg?logo=CreativeCommons&style=flat-square)](https://github.com/QuCAI-Lab/ibm2021-open-science-prize/blob/dev/LICENSE.md)
[![Contributions](https://img.shields.io/badge/contributions-welcome-orange?style=flat-square)](https://github.com/QuCAI-Lab/ibm2021-open-science-prize/pulls)
[![Maintenance](https://img.shields.io/badge/Maintained%3F-yes-green.svg)](https://github.com/QuCAI-Lab/ibm2021-open-science-prize/graphs/commit-activity)
[![Release](https://img.shields.io/github/release/QuCAI-Lab/ibm2021-open-science-prize.svg)](https://github.com/QuCAI-Lab/ibm2021-open-science-prize/releases)

<div align="center">
  <h1> <a href="https://research.ibm.com/blog/quantum-open-science-prize">2021 IBM Open Science Prize</a>  </h1>
  <h1><b> Supplementary Material </b></h1>
  <h3><b> Simulating the XXX Heisenberg Model Hamiltonian for a System of Three Interacting Spin-1/2 Particles on IBM Quantum’s 7-qubit Jakarta Processor </b></h3>
</div>
<br> 

<center> <b>Author: ¹Lucas Camponogara Viera</b></center>
<center>
<b><a target="_blank" href="https://en.ntnu.edu.tw/">¹National Taiwan Normal University - NTNU, Taipei, Taiwan</a></b>.
</center>

<br>

<table class="tfo-notebook-buttons" align="head">
  <td>
    <a target="_blank" href="https://github.com/QuCAI-Lab/ibm2021-open-science-prize"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /></a>
  </td>
</table>

In [2]:
%autosave 30

Autosaving every 30 seconds


---
In this Jupyter notebook, the reader will find some of the equations of the quantum simulation presented in the `simulation.ipynb` notebook file and beyond. The mathematical framework employed in the remainder of this Jupyter notebook is the contemporary matrix mechanics formalism of quantum mechanics initially developed by Werner Heisenberg, Max Born, and Pascual Jordan in a sequel of articles between 1925 and 1926. Within this formalism, the central pivots of the theory are the physical observables (e.g., spin, electric charge, magnetic flux, position, momentum, etc.) represented by Hermitian operators (self-adjoint matrices) where the corresponding eigenvectors form a set of orthonormal basis for the Hilbert space.

---

# Table of Contents

- **[Complex Analysis](#complex)**
  - Complex numbers
  - Complex functions

- **[Linear Algebra](#algebra)**
  - Matrices
  - Pauli Operators
  - Properties of complex and square matrices
  - Matrix exponential
  - Linear independence of eigenvectors
  - Diagonalization of Operators
- **[Postulates of quantum mechanics](#postulates)**
  - Postulate 1 (State Space)
  - Postulate 2 (Evolution)
  - Postulate 3 (Measurement)
  - Postulate 4 (Composite State)
- **[System's Dynamics: Schrödinger equation](#dynamics)**
 - Wave Mechanics formalism
 - Matrix Mechanics formalism
- **[Solution for the Schrödinger equation](#solutions)**
- **[Gate Algebra and Circuit Implementation](#gates)**
  - Gate Operations
  - General $U3(\theta, \phi, \lambda)$ single-qubit gate
  - General $R_{\hat{n}}(\theta)$ single-qubit gate
  - General $U(\alpha, \beta, \gamma, \delta)$ single-qubit gate
  - General General $C_n^{j}(U_{2^m})$ gate with $n$ control qubits and $m$ target qubits
  - Gate Identities
  - Gate Decomposition
  - Gate implementation with NumPy and Qiskit
- **[The Heisenberg XXX Spin-1/2 Lattice Model for  N=3  Three Particles](#model)**
- **[Time Evolution of the initial state  |110⟩  under  hheis3  according to  Uheis3(t=π)](#evolution)**
  - Probability Over Time  t.
- **[The Trotter-Susuki formula](#trotter)**
- **[Decomposition of  UHeis3(t)  using Trotterization](#decomp)**
- **[Transpilation of  UHeis3(t)  Into Quantum Gates](#transp)**
  - The  e−itZZ  gate
  - The  e−itXX  gate
  - The  e−itYY  gate
- **[State Fidelity](#fid)**
- **[References](#ref)**



# &nbsp; <a href="https://creativecommons.org/licenses/by/4.0/"><img valign="middle" src="https://img.icons8.com/copyright" width="50"></a> License

In [3]:
#@title Copyright 2022.
# This code is part of heisenberg-model.
#
# (C) Copyright NTNU QuCAI-Lab, 2022.
#
# This code is licensed under the Creative Commons Zero v1.0 Universal License. 
# You may obtain a copy of the License at https://github.com/QuCAI-Lab/ibm2021-open-science-prize/blob/dev/LICENSE.md.

# &nbsp; <a href="https://colab.research.google.com/"><img valign="middle" src="https://www.tensorflow.org/images/colab_logo_32px.png" width="50"></a> Pip Install

* **Run the following cells only if you are running this Jupyter notebook outside the [heisenberg_model environment](https://github.com/QuCAI-Lab/ibm2021-open-science-prize/blob/dev/environment.yml).**

Installing `Qiskit`, `NumPy`, and `pylatexenc`.

In [33]:
'''
# Installing a non-default library

!python3 -m pip install <library_name>

# Alternative 

!apt-get -qq install -y <library_name> && python3 -m pip install -U <library_name> 

# Upgrading an installed library

!python3 -m pip install -U --upgrade <library_name> 
'''

'\n# Installing a non-default library\n\n!python3 -m pip install <library_name>\n\n# Alternative \n\n!apt-get -qq install -y <library_name> && python3 -m pip install -U <library_name> \n\n# Upgrading an installed library\n\n!python3 -m pip install -U --upgrade <library_name> \n'

In [34]:
try:
  import pip, pkg_resources
  pkg_resources.require("pip>=21.1.3") # Latest version: 22.0.4
  print(pip.__version__)
except:
  import subprocess, sys
  cmd = "python3 -m pip install --upgrade pip"
  process = subprocess.Popen(cmd,shell=True,bufsize=1,stdout=subprocess.PIPE,
                             stderr=subprocess.STDOUT,encoding='utf-8',errors='replace') 
  while True:
    out = process.stdout.readline()
    if out == '' and process.poll() is not None:
      break
    if out:
      print(out.strip(), flush=False)
      sys.stdout.flush()
  #raise

21.1.3


In [35]:
#!python3 -m pip install pip==version_number # To downgrade pip.
!pip --version

pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


**NumPy:**

In [36]:
!python3 -m pip install numpy==1.20.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Qiskit:**

In [37]:
!python3 -m pip install qiskit==0.35.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**pylatexenc**:

In [38]:
# The 'pylatexenc' library is required to use 'MatplotlibDrawer'.

!python -m pip install pylatexenc==2.10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# &nbsp; <a href="#"><img valign="middle" height="45px" src="https://img.icons8.com/python" width="45" hspace="0px" vspace="0px"></a> Dependencies

- Importing modules.

In [4]:
import IPython
import qiskit
import numpy as np 
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit.circuit.library.standard_gates import U3Gate, RYGate
from qiskit.circuit import Parameter
import qiskit.quantum_info as qi
from qiskit import BasicAer

In [5]:
### If using Google Colab, one needs to restart Colab Runtime after pip install. ###

try:
  import pkg_resources, watermark
  pkg_resources.require("watermark>=2.3.0") # Force watermark version.
  print(watermark.__version__)
except:
  import subprocess, sys
  if "google.colab" in sys.modules:
    cmd = "python3 -m pip install --upgrade watermark" # Upgrade watermak.

    process = subprocess.Popen(cmd,shell=True,bufsize=1,stdout=subprocess.PIPE, \
                              stderr=subprocess.STDOUT,encoding='utf-8',errors='replace') 
    while True: 
      out = process.stdout.readline() # The first line of the file.
      if out == '' and process.poll() is not None: # Run the loop until condition is True.
        break 
      if out:
        print(out.strip(), flush=False) # Removes leading and trailing empty spaces. 
        sys.stdout.flush()
    #raise # To raise the import error. Upgrade will be successful regardless.

2.3.0


In [6]:
# If you get hit by the error "No module named watermark", run this cell twice!

#%load_ext watermark
%reload_ext watermark
%watermark -a 'LucasCamponogaraViera' -gu 'QuCAI-Lab' -ws 'https://github.com/QuCAI-Lab/ibm2021-open-science-prize' -w -u -d -v -m -iv

Author: LucasCamponogaraViera

Github username: QuCAI-Lab

Website: https://github.com/QuCAI-Lab/ibm2021-open-science-prize

Last updated: 2022-06-07

Python implementation: CPython
Python version       : 3.7.13
IPython version      : 7.34.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 165 Stepping 5, GenuineIntel
CPU cores   : 20
Architecture: 64bit

qiskit   : 0.35.0
watermark: 2.3.0
IPython  : 7.34.0
numpy    : 1.20.1

Watermark: 2.3.0



In [7]:
print(IPython.sys_info())

{'commit_hash': 'fcc71cc32',
 'commit_source': 'installation',
 'default_encoding': 'cp1252',
 'ipython_path': 'C:\\Users\\User\\anaconda3\\envs\\heisenberg-model\\lib\\site-packages\\IPython',
 'ipython_version': '7.34.0',
 'os_name': 'nt',
 'platform': 'Windows-10-10.0.19041-SP0',
 'sys_executable': 'C:\\Users\\User\\anaconda3\\envs\\heisenberg-model\\python.exe',
 'sys_platform': 'win32',
 'sys_version': '3.7.13 (default, Mar 28 2022, 08:03:21) [MSC v.1916 64 bit '
                '(AMD64)]'}


# Complex Analysis<a name="complex" />  

## Complex numbers



A complex number $z \in \mathbb{C}$ is an object defined as:

\begin{eqnarray}
z \equiv \Re(z)+i\Im(z),
\end{eqnarray}

where:

- $\Im(z) \in \mathbb{R}$ is the imaginary part.
- $\Re(z) \in \mathbb{R}$ is the real part.
- $i \equiv \sqrt{-1} \therefore i^2=-1$ is the imaginary unit.

**Conjugate**

The conjugate of a complex number is defined as:

\begin{eqnarray}
z^* \equiv \Re(z)-i\Im(z),
\end{eqnarray}

**Modulus**

The modulus of a complex number is defined as:

\begin{eqnarray}
|z| \equiv \sqrt{zz^*} &=& \sqrt{(\Re(z)+i\Im(z))(\Re(z)-i\Im(z))} \\ 
&=& \sqrt{\Re(z)\Re(z)-i\Re(z)\Im(z)+i\Im(z)\Re(z)-i^{2}\Im(z)\Im(z)} \\
&=& \sqrt{\Re(z)\Re(z)-i^{2}\Im(z)\Im(z)} \\ 
&=& \sqrt{\Re(z)^{2}+\Im(z)^{2}} \\ 
\therefore zz^* &=& |z|^2.
\end{eqnarray}


**Properties**

- 1) $z_1 \pm z_2=\Re(z_1\pm z_2)+i\Im(z_1\pm z_2)$.
- 2) $z_1z_2=\Re(z_1z_2)+i\Im(z_1z_2).$
- 3) $\frac{z_1}{z_2} = \Re(z_1/z_2)+i\Im(z_1/z_2).$
- 4) $z+z^* = 2 \Re(z)$.
- 5) $z-z^* = 2i \Im(z)$.
- 6) $(z_1\pm z_2)^*=z_1^* \pm z_2^*.$
- 7) $(z_1z_2)^*=z_1^*z_2^*.$
- 8) $|z_1z_2|=|z_1||z_2|.$
- 9) $\left|\frac{z_1}{z_2}\right|=\frac{|z_1|}{|z_2|}.$
- 10) $|z_1+z_2|^2 = |z_1|^{2}+|z_2|^{2}+ 2\bigg[\Re(z_1)\Re(z_2)+\Im(z_1)\Im(z_2)\bigg].$

**Complex Plane**

A complex vector (a.k.a phasor) denoted $\vec{z}$ in the bidimensional cartesian complex plane has the form:

\begin{eqnarray}
\vec{z} \equiv \Re(z) \hat{e}_1 + \Im(z)\hat{e}_2,
\end{eqnarray}
where:

- $\Re(z) \equiv x = |\vec{z}|\cos(\theta) \in \mathbb{R}$ is the x-axis coordinate.
- $\Im(z) \equiv y = |\vec{z}|\sin(\theta) \in \mathbb{R}$ is the y-axis coordinate.
- $\hat{e}_i$ denotes the right-handed orthonormal vector (a.k.a versor) satisfying $\hat{e}_i\cdot\hat{e}_j=\delta_{ij}$. One convention is $\hat{e}_i=\hat{i}$ and $\hat{e}_2 = \hat{j}$.

Adopting the aforementioned convention, the **polar form** of the **complex vector** takes the form:

\begin{eqnarray}
\vec{z} \equiv \Re(z) \hat{i} +\Im(z) \hat{j}=|\vec{z}|\cos(\theta)\hat{i}+|\vec{z}|\sin(\theta)\hat{j} = |\vec{z}|(\cos(\theta)\hat{i}+\sin(\theta)\hat{j}) \doteq r\hat{r}.
\end{eqnarray}

The **polar form** of the **complex number** takes the form:

\begin{eqnarray}
z \equiv \Re(z)+i\Im(z)=|\vec{z}|\cos(\theta)+i|\vec{z}|\sin(\theta) = |\vec{z}|(\cos(\theta)+i\sin(\theta))=|\vec{z}|e^{i\theta}.
\end{eqnarray}

And the modulus become:

$$|z|=|\vec{z}| = \sqrt{\vec{z} \cdot \vec{z}} = \sqrt{x^2+y^2}.$$


## Complex functions



The Taylor series around the point $x=x_0$ is defined as:

\begin{eqnarray}
g(x)|_{x_0}= \sum_{n=0}^{\infty} f^{n}(x_0)\frac{(x-x_0)^n}{n!},
\end{eqnarray}

where $f^{n}(x_0) = \frac{\partial^n f(x_0)}{\partial x^n}$ denotes the n-th derivative of $f(x)$ evaluated at the point $x=x_0$. In the particular case of $x_0=0$, the Taylor series becomes the [Maclaurin series](https://mathworld.wolfram.com/MaclaurinSeries.html).

From the above Taylor series, it is possible to expand any complex function $f:\mathbb{C}\rightarrow\mathbb{C}$. In particular, the complex exponential function with complex number $z \equiv x+iy$, can be expanded around $x_0=0$ in the form:

\begin{eqnarray}
e^{z}=\sum_{n=0}^{\infty}\frac{z^{n}}{n!} = 1+z+\frac{z^{2}}{2}+\frac{z^{3}}{3!}+\cdots+\frac{z^{n}}{n!},
\end{eqnarray}

which is not itself a polynomial, but the limit of a sequence of polynomials.

From the above result, the Euler formula is derived as:

\begin{eqnarray}
e^{i\theta}=\sum_{n=0}^{\infty}\frac{(i\theta)^{n}}{n!} &=& 1+(i\theta)+\frac{(i\theta)^{2}}{2}+\frac{(i\theta)^{3}}{3!}+\frac{(i\theta)^{4}}{4!}+\frac{(i\theta)^{5}}{5!}+\cdots+\frac{(i\theta)^{n}}{n!} \\
&=& \left(1-\frac{\theta^{2}}{2}+\frac{\theta^{4}}{4!}+\cdots+\right) +i\left(\theta-\frac{\theta^{3}}{3!}+\frac{\theta^{5}}{5!}+\cdots+\right) \\
&=& \cos(\theta)+i\sin(\theta).
\end{eqnarray}

For complex numbers $z_1, z_2 \in \mathbb{C}$ and real numbers $r_1, r_2 \in \mathbb{R}$, the following properties hold:

- 1) $e^{r_1+r_2} = e^{r_1} e^{r_2}$.
- 2) $e^{r_1+ir_2} = e^{r_1} e^{ir_2}$.
- 3) $e^{z_1+z_2} = e^{z_1} e^{z_2}$.
- 4) $z_1z_2=r_1e^{i\theta_1}r_2e^{i\theta_2}=r_1r_2e^{i(\theta_1+\theta_2)}$.


# Linear Algebra<a name="algebra" />  

## Matrices


**1)** A square matrix $A$ is **Invertible** (a.k.a nonsingular or nondegenerate) if there exists a square matrix $B$ such that:

$$AB=BA=I \implies B=A^{-1},$$

where $B$ is called the inverse matrix of $A$. Non-square matrices do not have an inverse.

**The invertible matrix theorem** *A matrix is non-invertible (a.k.a singular or degenerate) if, and only if, 0 (zero) is an eigenvalue of the matrix.*

Nonrigorous proof: since the product of the eigenvalues is equal to the matrix determinant, if one eigenvalue is zero, the determinant is zero, hence no inverse exists. The converse is also true: if a matrix has zero determinant, the matrix has no inverse.


**2)** A square matrix $A$ is **Involutory** if it satisfies:

$$A=A^{-1}\implies AA^{-1}=A^{2}=I.$$

The eigenvalues of an Involutory matrix are $\pm 1$. The determinant of an Involutory matrix over any field is $\pm 1$. A matrix that is both Hermitian and Unitary at the same time is an Involutory matrix. The converse is not true, i.e, a Normal matrix is not always Hermitian and Unitary, as one can find a counterexample. The 2x2 Pauli matrices are all Involutory.


**3)** A real-valued square matrix $A$ is **Orthogonal** if:

$$AA^T=A^TA =I \implies A^T=A^{-1}.$$ 

In the component form:

$$a_{ij}=(a^{-1})_{ji}.$$

An Orthogonal matrix is Invertible and Unitary over the field of the real numbers. These matrices preserve the inner product, i.e, they form a group denoted $O(n)$ of unitary transformations. The determinant of an Orthogonal matrix is $\pm 1$. The group of Orthogonal matrices with $det A=1$ is known as the special group $SO(n)$ of rotation matrices.


**4)** A complex-valued square matrix $U$ is **Unitary** if it satisfies:
    
$$U^{-1}=U^{\dagger}\implies U^{\dagger}U=UU^{\dagger}=UU^{-1}=I.$$

Furthermore, the row and column vectors of a Unitary matrix are orthonormal, i.e, they form an orthonormal basis. All the eigenvalues of a Unitary matrix have modulus (absolute value) equal to 1, i.e, they lie on the unit circle of the complex plane, hence they can be written in the form $\lambda=e^{i\theta}$ for some real-valued number $\theta$. A unitary matrix whose entries are all real numbers is an Orthogonal matrix. Moreover, Unitary matrices preserve the inner product.


**5)** A real-valued square matrix $A$ is **Symmetric** if:

$$A^T=A \implies a_{ji}=a_{ij}.$$


**6)** A complex-valued square matrix $H$ is **Hermitian** if:

$$H=H^{\dagger}.$$

The eigenvalues of a Hermitian matrix are all real. An Hermitian matrix whose entries are all real numbers is a Symmetric matrix.


**7)** A complex square matrix $A$ is **Normal** if it commutes with its adjoint (a.k.a conjugate transpose or Hermitian conjugate):

$$[A,A^{\dagger}]=AA^{\dagger}-A^{\dagger}A=\mathbb{O}.$$

Hermitian and Unitary matrices are always Normal. From the spectral theorem, all Normal matrices are diagonalizable and, therefore, Hermitian and Unitary matrices are as well. A Normal matrix is Unitary iff all of its eigenvalues (the matrix spectrum) lie on the unit circle of the complex plane, i.e, they have modulus 1. Moreover, a Normal matrix is Hermitian iff all its eigenvalues are real.


**8)** A square matrix $A$ is **Idempotent** if:

$$A^2=A.$$

**Spectral decomposition theorem for Normal matrices** *Any Normal operator $\hat{\mathcal{O}}$ on a $n$-dimensional vector space $V$ is unitarily diagonalizable with respect to some orthonormal basis for $V$. Conversely, any unitarily diagonalizable operator is normal.*

It follows that a Normal operator has a spectral decomposition of the form:

\begin{eqnarray}
\hat{\mathcal{O}} = \sum_{j=1}^n o_j |o_j\rangle \langle o_j|,
\end{eqnarray}
where $\{|o_j \rangle \}|_{j=1}^n$ is a basis set of $n$ linearly independent orthonormal eigenvectors of $\hat{\mathcal{O}}$ with eigenvalues $o_j$.

**Note1:** the above theorem does not imply that any diagonalizable operator is Normal, this is only true for unitary diagonalization. There are matrices which are not Normal, but are diagonalizable by orthogonal operators (orthogonally diagonalizable). See the `spectral theorem for Symmetric matrices` for more information.

**Note2:** more on matrix diagonalization (criteria and method) in the following section `"Diagonalization of Operators"`.

## Pauli Operators



The set of Pauli operators $\{\sigma_j\}|_{j=1}^3$ is a set of 2x2 complex-valued matrices that are each Hermitian, Unitary and, therefore, Involutory and Normal. Each matrix can be defined as:

\begin{eqnarray}
\sigma_j \equiv 
\begin{pmatrix}
\delta_{j3} & \delta_{j1} - i \delta_{j2} \\
\delta_{j1} + i \delta_{j2} & -\delta_{j3}
\end{pmatrix}.
\end{eqnarray}

Where $\delta_{jk}$ is the Kronecker delta defined as:

\begin{eqnarray}
\delta_{jk} \equiv \begin{cases}
0, & \mbox{if } j \ne k. \\
1, & \mbox{if } j=k. \end{cases}
\end{eqnarray}

With that:

\begin{eqnarray}
\sigma_{j=1} \equiv X &=& 
\begin{pmatrix}
0 & 1 \\
1 & 0
\end{pmatrix}, \\
\sigma_{j=2} \equiv Y &=&
\begin{pmatrix}
0 & -i \\
i & 0
\end{pmatrix}, \\
\sigma_{j=3} \equiv Z &=& 
\begin{pmatrix}
1 & 0 \\
0 & -1
\end{pmatrix} .
\end{eqnarray}




## Properties of complex and square matrices


For distinct square and complex matrices $A$ and $B$, identity matrix $I$, real valued numbers $\alpha$, $\beta \in \mathbb{R}$, and nonnegative integer $n \in \mathbb{Z}^{0+}$:

\begin{eqnarray}
A^0&=&I.\\
A^{\alpha}A^{\beta}&=&A^{\alpha+\beta} \text{ (Product of powers property)}.\\
(A^{\alpha})^\beta&=&A^{\alpha\beta}.\\
(AB)^n &=& \prod_{j=1}^n (AB)_j = (AB)_1(AB)_2\cdots (AB)_n, \forall n \in \mathbb{Z}^{0+}.\\
(AB)^n &=& A^n B^n = B^n A^n\text{ iff } [A,B]=\mathbb{O}.\\
(AB)^{\dagger}&=&B^{\dagger}A^{\dagger}.\\
det(AB)&=&det(BA)=det(A)det(B).\\
\end{eqnarray}

The adjoint operation is anti-linear:

$$\Big(\sum_j \lambda_j A_j \Big)^{\dagger} = \sum_j \lambda_j^{*} A_j^{\dagger}.$$

It follows that:

$$(A+B)^{\dagger}=A^{\dagger}+B^{\dagger}.$$

The following regards the integer power of an Involutory matrix. Given an Involutory matrix $A$ and an integer $n \in \mathbb{Z}^{0+}$:

\begin{eqnarray}
A^{2n}&=&(A^2)^n=I^n=I \text{ (even power)}, \\
A^{2n+1} &=& A \text{ (odd power)}.
\end{eqnarray}

Given a diagonal matrix $D$ and a unitary matrix $U$:

\begin{eqnarray}
(UDU^{\dagger})^n &=& UD(U^{\dagger}U)D(U^{\dagger}\cdots U)DU^{\dagger}=UD^n U^{\dagger}.
\end{eqnarray}

## Matrix exponential


The taylor series for a complex matrix $A$ reads:

\begin{eqnarray}
e^{A}=\sum_{n=0}^{\infty}\frac{A^{n}}{n!} = I +A+\frac{A^{2}}{2}+\frac{A^{3}}{3!}+\cdots+\frac{A^{n}}{n!}.
\end{eqnarray}

The corresponding Euler formula for matrix exponential considering a Involutory matrix $A$, such that $A^2=I$, is as follows:

\begin{eqnarray}
e^{\pm i\theta A} &=& I \pm i\theta A + \frac{(i\theta A)^2}{2!} + \frac{(\pm i\theta A)^3}{3!} + \frac{(i\theta A)^4}{4!} +  \frac{(\pm i\theta A)^5}{5!} + \cdots + \frac{(\pm i\theta A)^n}{n!}\\
&=& I \pm i\theta A - \frac{(\theta A)^2}{2!} \pm \frac{(i\theta A)^3}{3!} + \frac{(\theta A)^{4}}{4!} \pm \frac{(i\theta A)^5}{5!} + \cdots + \frac{(\pm i\theta A)^n}{n!} \\
&=& I \pm i\theta A - \frac{\theta^2}{2!}I \pm i \Bigg(\frac{-\theta^3}{3!}\Bigg) A + \frac{\theta^4}{4!} I \pm i \frac{\theta^5}{5!}A + \cdots +\\
&=&\left(1-\frac{\theta^{2}}{2}+\frac{\theta^{4}}{4!}+\cdots+\right) I \pm i\left(\theta-\frac{\theta^{3}}{3!}+\frac{\theta^{5}}{5!}+\cdots+\right)A\\
&=& \cos(\theta) I \pm i \sin(\theta)A.
\end{eqnarray}



### Properties:

- 1) For a diagonal matrix $D$ with eigenvalue $\lambda$ and eigenstate $|\psi\rangle$:

$$e^D|\psi\rangle=\sum_{n=0}^\infty \frac{D^n |\psi\rangle}{n!} = \sum_{n=0}^\infty \frac{\lambda^n |\psi\rangle}{n!}=e^{\lambda}|\psi\rangle.$$

Where $$D^n |\psi\rangle = D\cdots D |\psi\rangle = D \cdots \lambda |\psi\rangle = \lambda \cdots \lambda |\psi\rangle  = \lambda^n |\psi\rangle,$$

follows from the eigenvalue-eigenvector equation $D|\psi\rangle = \lambda |\psi\rangle$.



- 2) Given a diagonal matrix $D$ and a unitary matrix $U$:

\begin{eqnarray}
e^{\theta (UDU^{\dagger})}= Ue^{\theta D}U^{\dagger}.
\end{eqnarray}

*Proof:*

\begin{eqnarray}
e^{\theta (UDU^{\dagger})} &=& \sum_{n=0}^\infty \frac{(\theta U D U^{\dagger} )^n}{n!} \\
&=& \sum_{n=0}^\infty \frac{U \theta^n D^n U^{\dagger}}{n!}\\
&=& U \Bigg( \sum_{n=0}^\infty \frac{(\theta D)^n}{n!} \Bigg) U^{\dagger}\\
&=& Ue^{\theta D}U^{\dagger},
\end{eqnarray}

where it was used $(UDU^{\dagger})^n = UD(U^{\dagger}U)D(U^{\dagger}\cdots U)DU^{\dagger}=UD^n U^{\dagger}$.


- 3) Given a complex square matrix $A$ and identity matrix $I$:

\begin{eqnarray}
e^{A} &=& lim_{N\rightarrow \infty} \left(I+ \frac{A}{N}\right)^N \\
e^{A\otimes I} &=& e^{A}\otimes I.
\end{eqnarray}


- 4) If two square matrices $A$ and $B$ commute, i.e, $[A, B] = \mathbb{O}$:

\begin{eqnarray}
e^{iA}e^{iB} = e^{i(A+B)}.
\end{eqnarray}

- 5) If two square matrices $A$ and $B$ do not commute, i.e, $[A, B] \neq \mathbb{O}$:

\begin{eqnarray}
e^{iA}e^{iB} \neq e^{i(A+B)}, \\
e^{iA/n}e^{iB/n} \approx e^{i(A+B)/n},
\end{eqnarray}

for a small slice $n$ with approximation error scaling as $1/n^2$.


- 6) Jacobi’s formula:

$$ \det (e^{A}) = e^{tr(A)}.$$


- 7) Golden-Thompson inequality for Symmetric or Hermitian matrices $A$ and $B$:

$$tr (e^{(A+B)}) \leq tr (e^{A}e^{B}).$$

- 8) $e^\mathbb{O} = I.$

- 9) $(e^{A})^T = e^{(A^T)}.$

## Linear independence of eigenvectors

**Theorem** Let $A: V \rightarrow V$ be a linear operator (a mapping) with $A$ an $n$x$n$ square matrix on a space $V$ of dimension $\dim V(A)=n$. If $A$ has $n$ distinct eigenvalues, then it has a set of $n$ linearly independent eigenvectors that form a basis for $V$. 

**Definition (Algebraic multiplicity)** Let $A$ be an $n$x$n$ square matrix with $n$ possibly repeated eigenvalues $\lambda_1, \cdots, \lambda_n$ as solutions (root) of the characteristic polynomial equation

$$det(\lambda I-A) = (\lambda-\lambda_1) \cdot, ... , \cdot (\lambda - \lambda_n) = 0.$$

The algebraic multiplicity of an eigenvalue $\lambda_n$ denoted $\mu(\lambda_n) \in \mathbb{N}$ is the number of times it is a root of the characteristic polynomial equation.

**Definition (Geometric multiplicity)** Let $A$ be an $n$x$n$ matrix with some eigenvalue $\lambda_n$. The geometric multiplicity of $\lambda_n$ of $A$ associated with eigenspace $E_n$ and denoted $\dim E_n (A) \in \mathbb{N}$ is the number of linearly independent eigenvectors of $E_n$, i.e, the dimension (cardinality) of $E_n$.

**Defective eigenvalues:** when the geometric multiplicity is strictly less than the algebraic multiplicity.

**Non-defective eigenvalues:** when the geometric multiplicity is equal to the algebraic multiplicity.

**Proposition** If an $n$x$n$ square matrix $A$ has at least one defective eigenvalue, then it is impossible to find a set of $n$ linearly independent eigenvectors.

**Proposition** If an $n$x$n$ square matrix $A$ has repeated eigenvalues, and they are not defective, then it is possible to find a set of $n$ linearly independent eigenvectors.

## Diagonalization of Operators

- Criteria for diagonalizability, and eigendecomposition.


### Criteria for diagonalizability

**Proposition** An $n$x$n$ square matrix with $n$ distinct eigenvalues is always diagonalizable.

**Theorem** Let $A: V \rightarrow V$ be a linear operator (a mapping) with $A$ an $n$x$n$ square matrix on a space $V$ of dimension $\dim V(A)=n$. Each eigenvalue $\lambda_n$ of $A$ has a corresponding eigenspace $E_n$ of dimension $\dim E_n (A)$. $A$ is said to be diagonalizable if and only if the sum of the dimensions of its eigenspaces is equal to $n$, i.e, if there are $n$ linearly independent eigenvectors that form a basis for $V$.

The above theorem is valid over an arbitrary field, not limited to $\mathbb{R}$ and $\mathbb{C}$. Another equivalent definition is the following. An $n$x$n$ square matrix $A$ is diagonalizable iff

$$\dim E_n (A) = \mu(\lambda_n).$$ 

The above identity is voiced: the dimension of the eigenspace $E_n$ of $A$ corresponding to the eigenvalue $\lambda_n$ is equal to the algebraic multiplicity of $\lambda_n$ for every eigenvalue of $A$. 

Where:

- $\mu(\lambda_n)$ is the algebraic multiplicity of the eigenvalue $\lambda_n$, i.e, the number of times it is a root of the characteristic polynomial equation.
- $\dim E_{n} (A)$ is the geometric multiplicity of the eigenvalue $\lambda_n$ of $A$, i.e, the number of linearly independent eigenvectors of the associated eigenspace $E_n$, which is also the dimension (cardinality) of $E_n$. 

**Key takeaways:**

- A matrix **can be** diagonalized even if it has repeated eigenvalues.
- A matrix may be diagonalizable over the field $\mathbb{C}$, but may not be over the field $\mathbb{R}$.
- Diagonalizability **is not** related to invertibility, but rather to linear independence.


### Eigendecomposition

A diagonalizable square matrix $A$ can always be decomposed into a product of a diagonal matrix $D$ and a Unitary matrix $U$: 

$$A=UDU^{-1} \implies D = U^{-1}AU.$$

If A is Hermitian, then

$$A=UDU^{\dagger} \implies D = U^{\dagger}AU,$$

where $A = UDU^{\dagger}$ is known as the eigenvalue decomposition of $A$. The columns of $U$ are the eigenvectors of $A$, and $D$ is a diagonal matrix whose diagonal elements are the eigenvalues of $A$. 

The transformation $U^{\dagger}AU=D$ converts $A$ into a diagonal matrix $D$. Here, $U$ is the change-of-basis matrix that represents the operator $A$ in the canonical basis. The matrices $A$ and $D$ are similar in the sense that they represent the same operator in different basis.

This relation can be translated to the following definition: a linear operator $A: V \rightarrow V$ is diagonalizable if there is a basis of $V$ formed by the eigenvectors of $A$.

# Postulates of quantum mechanics<a name="postulates" />  

- Matrix Mechanics formalism using Dirac bra-ket notation.

## **Postulate 1 (State Space)**


Any **isolated (close)** quantum system is completely described by a normalized *state vector* $|\psi\rangle$ which is a unit vector embedded in a complex Hilbert space $|\psi\rangle \in \mathcal{H}$ known as the *state space* of the system endowed with an inner product $(\cdot,\cdot) \doteq \langle \cdot | \cdot \rangle$.

<br></br>

- **First case: Discrete Spectrum.**

When the spectrum of the eigenvalues are discrete, the state $|\psi\rangle$ of the quantum system is embedded in a Hilbert space $\mathcal{H}$ of dimension $d=\dim \mathcal{H}$. In a complete and discrete orthonormal basis $\{|o_j\rangle\}_{j=1}^d$, the state vector reads

\begin{eqnarray}
|\psi\rangle \doteq \sum_{j=1}^d c_j|o_j\rangle.
\end{eqnarray}

The inner product for orthonormal eigenstates becomes 

$$\langle o_j|o_k\rangle = \delta_{jk},$$

and the probability amplitude $c_j$ associated with the eigenstate $|o_j\rangle$ is obtained via the Fourier trick 

$$c_j=\langle o_j|\psi\rangle.$$ 

- **Second case: Continuous Spectrum.**

When the spectrum of the eigenvalues are continous, the state $|\psi\rangle$ of the quantum system is embedded in a infinite-dimensional Hilbert space $|\psi\rangle \in \mathcal{H}$  whose orthonormal basis $\{|q\rangle \}$ is formed by an uncountably infinite set of orthonormal vectors. In this particular case, the state of the quantum system in this basis reads:

\begin{eqnarray}
|\psi\rangle \doteq \int \psi_{q} |q\rangle d_{q},
\end{eqnarray}

which is a linear combination given in terms of an integral, rather than a sum (See Ref. [2], Section 3.2.3, pg. 103). The inner product thus becomes the Dirac delta distribution

\begin{eqnarray}
\langle q |q'\rangle  = \delta (q-q'), 
\end{eqnarray}

and the analogous Fourier trick gives the complex-valued scalar wave function $\psi_{q}$ as the probability amplitude:

\begin{eqnarray}
\psi_{q} = \langle q |\psi \rangle.
\end{eqnarray}

Note that $\psi_q$ is the standard wave function of the ubiquitous Schrodinger equation of wave mechanics, often denote by $\psi(q)$ or $\psi(x)$ if in the position space.

## **Postulate 2 (Evolution)**

The evolution of a **closed quantum system** over time is described by a *unitary transformation* given by the action of the propagator $\hat{U}_t\equiv\hat{U}(t)$ known as the Unitary time evolution operator. Therefore, the global state $|\psi_t\rangle$ of the system at time $t$ has evolved from the initial state $|\psi_0\rangle$ at time $t=0$ according to:

\begin{equation} 
|\psi_t\rangle=\hat{U}_t|\psi_0\rangle.
\end{equation}  

**Note:** in the particular case of closed systems, the evolution operator $\hat{U}_t$ of the system's global state $|\psi\rangle$ is Unitary, however, open quantum systems (systems interacting with its surroundings) described by mixed states, in general, have non-Unitary operators describing the evolution of its subsystems.







## **Postulate 3 (Measurement)**

Measurements of a quantum system are described by a collection \{$M_m$\} of *measurement operators* acting on the state space of the system with $m$ possible measurement outcomes. If the quantum system is prepared in a general state $|\psi\rangle$, the probability associated with a measurement outcome $o_m$ is:

\begin{equation} 
Pr(o_m)\doteq\langle \psi | M^{\dagger}_{m} M_{m} |\psi \rangle,
\end{equation} 

and the state of the system immediately after the measurement of the eigenvalue $o_m$ will be:

\begin{align} 
    |\psi_{o_m} \rangle= \frac{M_{m} |\psi \rangle}{\sqrt{\langle \psi | M^{\dagger}_{m} M_{m} |\psi \rangle}}. 
\end{align}

The measurement operators satisfy the completeness relation:

\begin{equation} 
\sum_{m} M_{m}^{\dagger} M_{m} = I,
\end{equation}

meaning that probabilities must sum to one, i.e, $|\psi\rangle$ is normalized:

\begin{equation} 
\sum_{m} Pr(o_m) = \sum_{m} \langle \psi | M^{\dagger}_{m} M_{m} |\psi \rangle = 1.
\end{equation}  

## **Postulate 4 (Composite State)**

Consider a quantum system composed of $N$ arbitrary subsystems, where subsystem $s$ is prepared in a qudit state $|\psi_{s}\rangle = \sum_{j=1}^{d_s} c_{j_s} |o_{j}\rangle_s$ embedded in a $d_s$-dimensional Hilbert space $\mathcal{H}_s^{d_s}$ with orthonormal basis $\{|o_{j}\rangle_s \}|_{j=1}^{d_s}$. The Hilbert state space $\mathcal{H}^{d}_{1\cdots N}$ of the composite physical system is given by the tensor product of its constituent Hilbert spaces (the state space of its subsystems or component physical systems): $\mathcal{H}^{d}_{1\cdots N}=\otimes_{s=1}^N \mathcal{H}_s^{d_s}=\mathcal{H}_1^{d_1} \otimes \mathcal{H}_2^{d_2} \otimes \cdots \otimes \mathcal{H}_N^{d_N}$, where $d=\prod_{s=1}^N d_s$ is the dimension of the composite space. And the corresponding orthonormal basis of the composite state space is obtained from the tensor product between the basis of each constituent state space: $\{|o_{j}\rangle \}|_{j=1}^{d}=\{|o_{j}\rangle_s \otimes \cdot\cdot\cdot \otimes|o_{k}\rangle_N \}|_{j,\cdots,k=1}^{d_s,\cdots,d_N}$. 

# System's dynamics: Schrödinger equation<a name="dynamics" />  

- **Wave Mechanics** and **Matrix Mechanics** formalisms.

## Wave Mechanics formalism


In the wave mechanics formalism of quantum mechanics, the dynamics of a system of particles is determined by the linear partial differential and time-dependent Schrödinger equation, which in the position representation reads:

\begin{eqnarray}
i \hbar \frac{\partial \Psi(\vec{r}, t)}{\partial t} &= \hat{H}(t)\Psi(\vec{r}, t).
\end{eqnarray}

Where: 

- $\Psi(\vec{r}, t) \doteq \Psi(x, y, z, t): \mathbb{R}^{4}\rightarrow\mathbb{C}$ is the complex valued scalar wavefunction containing the information about the particle's dynamics in the *position space* of continuous spectrum. The spectrum is the set of all eigenvalues of a given operator.
- $\hat{H}(t)$ is the Hamiltonian of the system, i.e, the sum of its kinetic and potential energy.
- $i$ is the imaginary unit.
- $\hbar$ is the reduced Planck constante a.k.a quantum of action.

The Schrödinger equation can also be written in the form:

\begin{eqnarray}
{i\hbar\partial_{t}\Psi(\vec{q},t)=\left(-\frac{\hbar^{2}}{2m} \nabla^{2}_{\vec{q}}+V(\vec{q},t) \right) \Psi(\vec{q},t)}.
\end{eqnarray}

Where:

- $\partial_t \equiv \frac{\partial}{\partial_t}$.
- $\nabla^2_{\vec{q}}$ is the second-order differential Laplace operator in generalized coordinates $q_j|_{j=1}^3$. 
- $V(\vec{q}, t)$ is the time-dependent potential energy.

The Laplace operator is defined as the divergence ($\vec{\nabla} \cdot $) of the gradient ($\vec{\nabla} \Phi$) and, therefore, it is a scalar operator and contrasts with the gradient which is a vector operator acting on a scalar field (function) $\Phi$. 

- The gradient operator in **generalized coordinates**. Given a scalar field $\Phi=\Phi(q_1, q_2, q_3)$,

$$\vec{\nabla}_{\vec{q}} \Phi = \frac{\partial \Phi}{h_j\partial q_j} \hat{e}_j =  \frac{1}{h_1}\frac{\partial \Phi}{\partial q_1} \hat{e}_1 + \frac{1}{h_2}\frac{\partial \Phi}{\partial q_2} \hat{e}_2 + \frac{1}{h_3}\frac{\partial \Phi}{\partial q_3} \hat{e}_3.$$

- The divergence operator in **generalized coordinates**. Given a vectorial field $\vec{A}=A(q_1, q_2, q_3)= A_1 \hat{e}_1+A_2 \hat{e}_2+A_3 \hat{e}_3$,

$$\vec{\nabla}_{\vec{q}} \cdot \vec{A} =  \frac{1}{h_1 h_2 h_3} \bigg[\frac{\partial (A_1 h_2 h_3)}{\partial q_1} + \frac{\partial (A_2 h_1 h_3)}{\partial q_2} + \frac{\partial (A_3 h_1 h_2)}{\partial q_3} \bigg].$$


- The Laplace operator in **Cartesian coordinates**. Given a scalar field $\Phi=\Phi(x, y, z)$,

$$\nabla^2_{\vec{r}} \Phi = \vec{\nabla} \cdot \vec{\nabla}(\Phi) = \partial_{rr} \Phi = \sum_{j=1}^3\frac{\partial^2 \Phi}{\partial r_j^2} = \frac{\partial^2 \Phi}{\partial x^2}+ \frac{\partial^2 \Phi}{\partial y^2} + \frac{\partial^2 \Phi}{\partial z^2}.$$

## Matrix Mechanics formalism


In the Hilbert state space (statevector) formalism (a.k.a qubit representation), the time-dependent Schrödinger equation reads:

\begin{eqnarray}
i \hbar\frac{d}{dt}|\psi(t)\rangle= \hat{H} |\psi(t)\rangle,
\end{eqnarray}

where $|\psi\rangle$ is the state of the quantum system in a linear superposition (combination) according to definitions for either a continuous or a discrete case as provided in the section `"Postulates of quantum mechanics"`.


### Schrödinger Picture (S-P)

In the **Schrödinger picture** (S-P) (representation), the evolution of the initial state $|\psi_{t=0}^S\rangle\equiv|\psi(t=0)\rangle$ over a time $t$ is given by:

\begin{equation} 
|\psi_t^S\rangle=\hat{U}_t|\psi_0^S\rangle. \tag{1}
\end{equation}  

While the operator describing a certain physical observable remains constant in time:

\begin{equation} 
\hat{O}_t^S=\hat{O}_0^S=\sum_{j=1}^d o_j |o_j^S\rangle\langle o_j^S|. \tag{2}
\end{equation}  

### Heisenberg Picture (H-P)

In the **Heisenberg picture** (H-P), the operator describing a certain physical observable evolves over a time $t$ according to:

\begin{eqnarray} 
\hat{O}_t^H &=& \sum_{j=1}^d o_j |o_j^H (t)\rangle\langle o_j^H (t)| = \sum_{j=1}^d o_j \hat{U}_{t}^{\dagger}|o_j^S\rangle\langle o_j^S|\hat{U}_t = \hat{U}_t^{\dagger}\sum_{j=1}^d o_j|o_j^S\rangle\langle o_j^S|\hat{U}_t \\
&=& \hat{U}_t^{\dagger} \hat{O}_0^S \hat{U}_t.
\end{eqnarray} 

where

\begin{equation} 
|o_j^H(t)\rangle = \hat{U}_t^{\dagger}|o_j^S\rangle. \tag{3}
\end{equation}  

While the global state $|\psi\rangle$ of the quantum system remains constant:

\begin{equation} 
|\psi_t^H\rangle = |\psi_0^S\rangle. \tag{4}
\end{equation}  

Regardless of the picture used, H-P or S-P, solution for $\hat{U}_t$ is obtained solving the Schrödinger equation of the time evolution operator:

\begin{equation} 
i\hbar \partial_t \hat{U}_t = \hat{H}_t\hat{U}_t. \tag{5}
\end{equation}  

From equations (1) and (5), one can verify the equivalent Schrödinger equation for the quantum state, as follows:

\begin{equation} 
i\hbar \partial_t |\psi_t\rangle = i\hbar \partial_t (\hat{U}_t |\psi_0\rangle) = (i\hbar \partial_t \hat{U}_t)|\psi_0\rangle = \hat{H}_t \hat{U}_t|\psi_0\rangle= \hat{H}_t|\psi_t\rangle. \tag{6}
\end{equation}  

Where:
- $\partial_t \equiv \frac{\partial}{\partial_t}$.

# Solution for the Schrödinger equation<a name="solutions" />  

- Matrix exponentiation.

Solution for the Schrödinger equation of the time-dependent evolution operator $\hat{U}(t)$ depends mostly on the characteristic of the Hamiltonian $\hat{H}$. In the matrix mechanics formalism, the Sch. eq. in the S-P writes

\begin{equation} 
i\hbar \frac{d}{dt}(\hat{U}(t) |\psi(0)\rangle)=\hat{H}(t) (\hat{U}(t) |\psi(0)\rangle).
\end{equation}  

One then has:

\begin{equation} 
 \frac{d}{dt} \hat{U}(t) = \frac{-i}{\hbar}\hat{H}\hat{U}(t).
\end{equation}  

A know fact from linear algebra is that any Unitary operator can be constructed by means of some Hermitian operator and a real number $\gamma$:

\begin{eqnarray}
\hat{U}=e^{-i\gamma\hat{H}}.
\end{eqnarray}

To show the above is true, one can prove its unitarity:

\begin{eqnarray}
\hat{U}^{\dagger}&=&(e^{-i\gamma\hat{H}})^{\dagger}=e^{i\gamma\hat{H}^{\dagger}} = e^{i\gamma\hat{H}}, \\
&\implies&
\hat{U}\hat{U}^{\dagger}=e^{-i\gamma\hat{H}}e^{i\gamma\hat{H}}=I,
\end{eqnarray}

where it was used the property: $(e^{z\hat{A}})^{\dagger} = e^{(z\hat{A})^{\dagger}}=e^{\hat{A}^{\dagger}z^{\dagger}}=e^{z^{\dagger}\hat{A}^{\dagger}}$, for a complex number $z$ and matrix $\hat{A}$.

In the particular case of a **time-independent Hamiltonian (conservative system)**, the above relation can be used to write the closed-form solution as:

\begin{eqnarray}
\hat{U}(t) = e^{-i\hat{H}t / \hbar}.
\end{eqnarray}

One can check this result by expanding the exponential function in Taylor series around the point $x_0=0$ (Maclaurin series) and differentiating term by term. The Maclauring series for $f(x)=e^{i\gamma\hat{H}}$ with $e^{\displaystyle{0}}=I$ is:

\begin{eqnarray}
e^{i\gamma\hat{H}} = \sum_{n=0}^{\infty} \frac{({i\gamma\hat{H}})^n}{n!} = I+i\gamma \hat{H}+\frac{(i\gamma \hat{H})^2}{2}+\frac{(i\gamma \hat{H})^3}{3!}+\cdots +\frac{(i\gamma \hat{H})^n}{n!}.
\end{eqnarray}

So that for $\gamma=-t/\hbar$:

\begin{eqnarray}
\frac{d}{dt} (e^{-i\hat{H}t / \hbar})&=&\frac{d}{dt} \left( I - i\hat{H}t/\hbar - \frac{\hat{H}^2 t^2}{2\hbar^2}  + \cdots + \frac{(-i\hat{H}t)^n}{n!\hbar^n}\right)  \\
&=& \left(0 -i\hat{H}/\hbar - \frac{\hat{H}^2t}{\hbar^2} + \cdots + \frac{n(-i\hat{H})^nt^{n-1}}{n!\hbar^n}\right), \\
&=&  \frac{-i}{\hbar}\hat{H} \left(I - \frac{i\hat{H}t}{\hbar} + \cdots + \frac{(-i\hat{H})^{n-1}t^{n-1}}{(n-1)!\hbar^{n-1}}\right), \text{ defining } k \equiv n-1\\
\\
&=&  \frac{-i}{\hbar}\hat{H} \left(I - \frac{i\hat{H}t}{\hbar} + \cdots + \frac{(-i\hat{H}t)^{k}}{k!\hbar^{k}}\right)\\
&=& \frac{-i}{\hbar}\hat{H}e^{-i\hat{H}t / \hbar},
\end{eqnarray}

where it was used: $\frac{n}{n!}=\frac{n}{n(n-1)!}=\frac{1}{(n-1)!}$.

Note that for $\hat{H}=\sum_{j}^n \hat{H}_j$, in general, $[\hat{H}_j, \hat{H}_k] \neq 0$, and thus $e^{-i\hat{H} \cdot t} \neq \prod_j e^{-i\hat{H}_j \cdot t}$. That is to say:

$$e^{-i\hat{H}t} = e^{-i\hat{H}_1\cdot t} \cdot e^{-i\hat{H}_2\cdot t} \cdots e^{-i\hat{H}_n\cdot t} \text{ iff } [\hat{H}_j, \hat{H}_k]=0.$$

Another way of looking at the problem of solving differential equations is to consider the first order approximation (see Ref. [1], Sec. 4.7.2):

\begin{eqnarray}
|\psi(t+\Delta t)\rangle &\approx& |\psi(t)\rangle + \Delta t \frac{d}{dt} |\psi(t)\rangle\\
&=& (I-\frac{i}{\hbar}\hat{H}\Delta t)|\psi(t)\rangle.
\end{eqnarray} 

The approximation to high order can be obtained considering small time evolutions ($t<<1$):

\begin{eqnarray}
|\psi(t)\rangle &\approx& (I-\frac{i}{\hbar}\hat{H} t)|\psi(0)\rangle \\
&=& (I-\frac{i t}{2\hbar}\hat{H} )(I-\frac{it}{2\hbar}\hat{H})|\psi(0)\rangle.
\end{eqnarray} 

Such that 
\begin{eqnarray}
|\psi(t)\rangle &=& lim_{N\rightarrow \infty} (I-\frac{it}{N\hbar}\hat{H})^N|\psi(0)\rangle \\ 
&=& e^{-i\hat{H}t/\hbar} |\psi(0)\rangle.
\end{eqnarray} 

Where it was used: $e^{\hat{A}} = lim_{N\rightarrow \infty} (I+ \frac{\hat{A}}{N})^N$.

# Gate Algebra and Circuit Implementation<a name="gates" />  

The bare minimum with implementations in NumPY & Qiskit.

Quantum gates are all represented by Unitary matrices, but they might not be Hermitian. Most frequently used quantum gates are Hermitian:

- [The Hadamard gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.HGate.html) (a.k.a H or superposition gate), induces a $\pi$ rotation about the $X+Z$ axis;
- [The CNOT gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CXGate.html) (a.k.a $CX$);
- [The SWAP gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.SwapGate.html);
- [The Toffoli gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CCXGate.html) (a.k.a CCNOT or Deutsch);
- Fredkin (CSWAP) gate;
- All Pauli gates: [X](https://qiskit.org/documentation/tutorials/circuits/3_summary_of_quantum_operations.html#X:-bit-flip-gate), [Y](https://qiskit.org/documentation/tutorials/circuits/3_summary_of_quantum_operations.html#Y:-bit--and-phase-flip-gate), [Z](https://qiskit.org/documentation/tutorials/circuits/3_summary_of_quantum_operations.html#Z:-phase-flip-gate). 

However, there are non-hermitian gates such as:

- The [phase gate T](https://qiskit.org/documentation/stubs/qiskit.circuit.library.TGate.html) (a.k.a pi/8 or fourth-root of Pauli-$Z$);
- The [phase gate S](https://qiskit.org/documentation/stubs/qiskit.circuit.library.SGate.html) a.k.a P gate (square-root of Pauli-$Z$).


A universal gate set is such to which any possible gate operation can be reduced to a finite sequence of gates from the set.

- The gate set containing the standard Non-clifford rotation gate operators $R_x(\theta)$, $R_y(\theta)$, $R_z(\theta)$, the phase shift gate $P(φ)$ and the CNOT gate is a common universal set of quantum gates.

- The Clifford + phase T (pi/8) gate set denoted {H, S, CNOT, T} is another widely used universal gate set. (See Ref. [1], Chap. 10 for fault-tolerant constructions of those gates).

Due to [Gottesman–Knill theorem](https://en.wikipedia.org/wiki/Gottesman%E2%80%93Knill_theorem), the Clifford set can be efficiently simulated in a classical computer and, therefore, is the main gate set used to approximate a target quantum circuit in the [CDR error mitigation technique](https://mitiq.readthedocs.io/en/latest/guide/cdr-4-low-level.html). 

In [13]:
import numpy as np

'''Defining Gates.'''

zero = np.array([1,0]) # |0>.
one = np.array([0,1]) # |1>.
outer0 = np.outer(zero, zero) # |0><0|.
outer1 = np.outer(one, one) # |1><1|.
sigma0 = np.identity(2) # Identity.
sigma1 = np.array([[0,1],[1,0]]) # Pauli-X.
sigma2 = np.array([[0,-1j],[1j,0]]) # Pauli-Y.
sigma3 = np.array([[1,0],[0,-1]]) # Pauli-Z.
had = (1/np.sqrt(2))*np.array([[1,1],[1,-1]]) # Hadamard.
CNOT=np.kron(sigma0, outer0) + np.kron(sigma1, outer1) # CNOT gate.

In [14]:
'''
Checking whether some Gates are Unitary and Hermitian.

Let * denote the complex conjugate (dagger).
Hermitian: A=A*.  
Unitary: AA*=A*A=I.
If a complex matrix A is both Unitary and Hermitian, then AA*=AA=I (Involutory).
'''

# np.dot(gate, gate) == gate@gate # >> True

try:
  if all(((gate@gate).round() == sigma0).all() for gate in [sigma1, sigma2, sigma3, had]) and (CNOT@CNOT == np.identity(4)).all():
    print('The X, Y, Z, H, and CNOT gates have been verified to be Unitary and Hermitian!')
  else:
    print('Gate error!')
except:
  print("Bad coding practice!")

The X, Y, Z, H, and CNOT gates have been verified to be Unitary and Hermitian!


## Gate Operations

### Pauli-Z gate

Let $Z$ denote the Pauli-$Z$ operator. Let $|0\rangle$ and $|1\rangle$ be column vectors (kets), each one representing the state of a classical bit. The action of the $Z$ gate on each state is:

$$ Z |0\rangle =  |0\rangle,$$
$$ Z |1\rangle =  -|1\rangle.$$

This means that 

$$|0\rangle = \begin{bmatrix} 1 \\ 0 \end{bmatrix},$$

and 

$$|1\rangle = \begin{bmatrix} 0 \\ 1 \end{bmatrix}, $$

are the two eigenstates of the $Z$-gate, i.e, they form the ubiquitous orthonormal basis set known as canonical basis or computational basis. Why? Because they satisfy the eigenvalue-eigenvector equation:

$$\hat{O} |o_j\rangle = o_j |o_j\rangle.$$

One can observe that the action of the $Z$ gate on state $|1\rangle$ induces a phase factor represented by the minus sign. Due to this phase factor, the $Z$ gate is also known as the "phase flip matrix".

**Note: the phase flip matrix should not be confused with the phase gate, which is a totally different gate.**

Other useful states are the following superposition states defined as linear combination (superposition) of classical bits $|0\rangle$ and $|1\rangle$:

$$|+\rangle \equiv \frac{1}{\sqrt{2}}(|0\rangle+|1\rangle) = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 \\ 1 \end{bmatrix}.$$

$$|-\rangle \equiv \frac{1}{\sqrt{2}}(|0\rangle-|1\rangle) = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 \\ -1 \end{bmatrix}.$$

Such that

$$Z |+\rangle \equiv \frac{1}{\sqrt{2}}(Z|0\rangle+Z|1\rangle) =\frac{1}{\sqrt{2}}(|0\rangle-|1\rangle)  = |-\rangle,$$
$$Z |-\rangle \equiv \frac{1}{\sqrt{2}}(Z|0\rangle-Z|1\rangle) = \frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)=|+\rangle.$$

In [15]:
# Implementation

sigma3@zero, sigma3@one # Z|0>, Z|1>

(array([1, 0]), array([ 0, -1]))

In [16]:
sigma3@(had@zero), sigma3@(had@one) # Z|+>, Z|->

(array([ 0.70710678, -0.70710678]), array([0.70710678, 0.70710678]))

### Hadamard gate

Let $H$ denote the Hadamard operator. The action of the Hadamard gate $H$ is as follows:

$$H |0\rangle = \frac{1}{\sqrt{2}}\Big(|0\rangle+|1\rangle\Big) = |+\rangle.$$
$$H |1\rangle = \frac{1}{\sqrt{2}}\Big(|0\rangle-|1\rangle\Big) = |-\rangle.$$
\begin{eqnarray}
H |+\rangle &=& \frac{1}{\sqrt{2}}(H|0\rangle+H|1\rangle)=\frac{1}{2}\Big[\Big(|0\rangle+|1\rangle \Big)+ \Big(|0\rangle-|1\rangle\Big)\Big]=|0\rangle,\\
&=& H(H|0\rangle) = |0\rangle.
\end{eqnarray}
\begin{eqnarray}
H|-\rangle &=&\frac{1}{\sqrt{2}} (H|0\rangle-H|1\rangle)=\frac{1}{2}\Big[\Big(|0\rangle+|1\rangle\Big)-\Big(|0\rangle-|1\rangle\Big)\Big]=|1\rangle,\\
&=& H(H|1\rangle) = |1\rangle.
\end{eqnarray}

Therefore, the Hadamard gate transforms the state of the qubit between the $X$ and $Z$ bases. Moreover, due to the first two relations, the Hadamard gate is also known as "superposition gate".

In [17]:
# Implementation

had@zero, had@one, had@(had@zero), had@(had@one) # H|0>, H|1>, H|+>, H|->

(array([0.70710678, 0.70710678]),
 array([ 0.70710678, -0.70710678]),
 array([1., 0.]),
 array([0., 1.]))

### Pauli-X gate

The action of the Pauli-$X$ gate is:

$$ X |0\rangle = |1\rangle.$$ 
$$X |1\rangle = |0\rangle.$$  

Furthermore:

$$X |+\rangle = \frac{1}{\sqrt{2}}(X|0\rangle+X|1\rangle)=\frac{1}{\sqrt{2}}(|1\rangle+|0\rangle)=|+\rangle.$$
$$X |-\rangle = \frac{1}{\sqrt{2}}(X|0\rangle-X|1\rangle) =\frac{1}{\sqrt{2}}(|1\rangle-|0\rangle)=-|-\rangle.$$

This means that $|+\rangle$ and $|-\rangle$ are two eigenstates of the $X$-gate, i.e, they form an orthonormal basis set a.k.a the $X$-basis.

In [18]:
# Implementation

sigma1@zero, sigma1@one, sigma1@(had@zero), sigma1@(had@one) # X|0>, X|1>, X|+>, X|->

(array([0, 1]),
 array([1, 0]),
 array([0.70710678, 0.70710678]),
 array([-0.70710678,  0.70710678]))

### Pauli-Y gate

The action of the Pauli-$Y$ gate is:

$$ Y |0\rangle = i|1\rangle.$$ 
$$ Y |1\rangle = -i|0\rangle.$$  

The two eigenstates of the $Y$-gate are:

$$|+y\rangle \equiv |\oplus\rangle = \frac{1}{\sqrt{2}}(|0\rangle+i|1\rangle) = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 \\ i \end{bmatrix},$$

and

$$|-y\rangle \equiv |\ominus\rangle = \frac{1}{\sqrt{2}}(|0\rangle-i|1\rangle) = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 \\ -i \end{bmatrix}.$$


In [19]:
# Implementation

sigma2@zero, sigma2@one # Y|0>, Y|1>.

(array([0.+0.j, 0.+1.j]), array([0.-1.j, 0.+0.j]))

## General $U3(\theta, \phi, \lambda)$ single-qubit gate



The three-parameter single qubit $U3(\theta, \phi, \lambda)$ gate reads:

\begin{equation}
U3(\theta, \phi, \lambda) = \begin{bmatrix} \cos(\theta/2) & -e^{i\lambda}\sin(\theta/2) \\ e^{i\phi}\sin(\theta/2) & e^{i\lambda+i\phi}\cos(\theta/2) \end{bmatrix},
\end{equation}

where:

  - $0\leq\lambda < 2\pi$ represents a phase.
  - $0\leq \phi < 2\pi$ changes the coherence.
  - $0\leq \theta \leq \pi$ performs the rotation around a given axis.

Particular gates:
  - $X = U3(\pi, 0, \pi)$.
  - $Y = U3(\pi, \pi/2, \pi/2)$.
  - $Z = U3(\pi,0,0)$.
  - $H=U(\pi/2,0,\pi)$.
  - $P=U(0,0,\lambda)$.
  - $R_x(\theta) = U3(\theta, -\pi/2,\pi/2)$.
  - $R_y(\theta) = U3(\theta, 0,0)$.
  - $R_z(\phi) = U3(0, \phi,0)$.



## General $R_{\hat{n}}(\theta)$ single-qubit gate

Given an operator $A=\hat{n}\cdot \hat{\sigma}$ and a real valued Euler angle $\gamma = \theta/2$, any single qubit standard rotation gate can be defined as:

$$ R_{\hat{n}}(\theta) \equiv e^{\pm i(\hat{n} \cdot \vec{\sigma})\theta /2}= \cos(\theta/2) I \pm i \sin(\theta/2)(\hat{n}\cdot \hat{\sigma}).$$

Where:
- $\hat{n}=n_x\hat{e}_x, n_y\hat{e}_y, n_z\hat{e}_z$ is a 3-dimensional unit vector.
- $\hat{e}_i$ denotes the right-handed orthonormal vector (a.k.a versor) satisfying $\hat{e}_i\cdot\hat{e}_j=\delta_{ij}$.
- $\vec{\sigma}=\sigma_1 \hat{e}_x + \sigma_2 \hat{e}_y + \sigma_3 \hat{e}_z$ is the three-component Pauli vector with $\sigma_j|_{j=1}^3 \in \{X, Y, Z\}$.
- $X$, $Y$, and $Z$ are the 2x2 Pauli operators (Hermitian, involutory and unitary matrices).
- $\hat{n}\cdot \vec{\sigma}=n_xX+n_yY+n_zZ$.
- $i^2=-1$ is the imaginary unit.

Most used single qubit standard rotation gates (non-Clifford gates):

\begin{equation}
R_x(\theta)=R_{(1,0,0)}(\theta)= \exp(\pm iX\theta/2) = cos(\theta/2)I\pm isin(\theta/2)X = \begin{bmatrix} \cos(\theta/2) & \pm i\sin(\theta/2) \\ \pm i\sin(\theta/2) & \cos(\theta/2) \end{bmatrix},
\end{equation}

\begin{equation}
R_y(\theta)=R_{(0,1,0)}(\theta) = \exp(\pm iY\theta/2) = cos(\theta/2)I\pm isin(\theta/2)Y = \begin{bmatrix} \cos(\theta/2) & \pm\sin(\theta/2) \\ \mp \sin(\theta/2) & \cos(\theta/2) \end{bmatrix},
\end{equation}

\begin{equation}
R_z(\phi)=R_{(0,0,1)}(\theta) = \exp(\pm iZ\phi/2) = cos(\theta/2)I\pm isin(\theta/2)Z = \begin{bmatrix} e^{\pm i\phi/2} & 0 \\0 & \mp e^{i\phi/2}
\end{bmatrix}.
\end{equation}

## General $U(\alpha, \beta, \gamma, \delta)$ single-qubit gate


Any arbitrary single qubit unitary gate can be written in the form:

\begin{align}
U(\theta) &= e^{i\alpha}R_{\hat{n}}(\theta)=e^{i\alpha} e^{-i\theta (\hat{n} \cdot \vec{\sigma})/2}\\
&=e^{i\alpha} \left(cos\left(\frac{\theta}{2}\right)I_2-sin\left(\frac{\theta}{2}\right)(\hat{n}\cdot \vec{\sigma})\right) \\
&= \begin{bmatrix} \cos(\theta/2)-in_z\sin(\theta/2) & -n_y \sin(\theta/2)-in_x\sin(\theta/2) \\ n_y\sin(\theta/2)-in_x\sin(\theta/2) & \cos(\theta/2)+in_z\sin(\theta/2) \end{bmatrix}.
\end{align}


Moreover, a general single qubit unitary gate can also be written as:

\begin{eqnarray}
  U(\alpha, \beta, \gamma, \delta) &=& e^{i\alpha}R_{\hat{n}}(\beta)R_{\hat{m}}(\gamma)R_{\hat{n}}(\delta), \\
  U &=& \begin{bmatrix} e^{i(\alpha-\beta/2-\delta/2)} \cos(\gamma/2) & -e^{i(\alpha-\beta/2+\delta/2)}\sin(\gamma/2) \\ e^{i(\alpha+\beta/2-\delta/2)}\sin(\gamma/2) & e^{i(\alpha+\beta/2+\delta/2)}\cos(\gamma/2) \end{bmatrix}.
\end{eqnarray}

Where:
- $R_{\hat{n}}$ is the standard rotation gate (non-Clifford gate).
- $\alpha$, $\beta$, $\gamma$, $\delta$ and $\theta \in \mathbb{R}$ are real-valued constants (Euler angles).


## General $C_n^{j}(U_{2^m})$ gate with $n$ control qubits and $m$ target qubits



A generic single qubit rotation [$CU(\theta, \phi, \lambda, \gamma)$](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CUGate.html) gate can be written as:

\begin{equation}
I\otimes |0\rangle\langle 0\rangle + e^{i \gamma } U(\theta, \phi, \lambda, \gamma) \otimes |1 \rangle \langle 1|,
\end{equation}

where $e^{i \gamma }$ is a global phase.

Let $U_{2^m}$ be a $2^m$ x $2^m$ unitary matrix and $I_{2^m}$ be a $2^m$ x $2^m$ identity matrix, then a generic multicontrolled gate with $n$ control qubits and $m$ target qubits is given by:

\begin{equation}
C_n^{j}(U_{2^m}) = \sum_{i=0, i\neq j}^{2^n-1}I_{2^m} \otimes |i\rangle\langle i|+ U_{2^m} \otimes |j\rangle\langle j|.
\end{equation}

Particular cases (Qiskit's little endian convention):

- $CU3(\theta, \phi, \lambda) \equiv C_1(U_{2})$ where $CRY = CU3(\theta, 0,0)$.

- (CNOT Gate) $CX \equiv C_1(U_{2}) = I\otimes |0\rangle\langle 0\rangle + X \otimes |1 \rangle \langle 1|$. The action of the CX gate can be represented by: $|a,b\rangle \rightarrow |a,a\oplus b\rangle$.

- (Toffoli Gate) $CCX \equiv C_2(U_{2}) = I\otimes I\otimes |0\rangle\langle 0\rangle + CX \otimes |1 \rangle \langle 1|$.

- $CCCX \equiv C_3(U_{2}) = I\otimes I\otimes I\otimes |0\rangle\langle 0\rangle + CCX \otimes |1 \rangle \langle 1|$.

The $\oplus$ symbol denotes the classical exclusive-OR (XOR) operation (a.k.a direct sum).

In textbook convention, for example, the CX (CNOT) gate reads: $|0\rangle\langle 0|\otimes I +|1\rangle\langle 1|\otimes X$.

## Gate Identities



- 1) $X^2=Y^2=Z^2=-iXYZ=I$ (Involutory square matrices).
- 2) $ZX=iY=-XZ$ (Anti-commutation).
- 3) $ZXZ=-X$.
- 4) $ZYZ=-Y$.
- 5) $HXH=Z$.
- 6) $HZH=X$.
- 7) $SXS^{\dagger}=Y$.
- 8) $SYS^{\dagger}=-X$.
- 9) $SZS^{\dagger}=Z$.


- 10) $Z=P(\pi)$.
- 11) $S=Z^{1/2} = P(\pi/2)$.
- 12) $S^2=SS=Z^{1/2}Z^{1/2}=Z = P(\pi)$.
- 13) $T=S^{1/2}=Z^{1/4}=P(\pi/4)$.

For $\sigma_j\in \{X,Y,Z\}$:
- 14) $R_{j}(\theta)= e^{\pm i\frac{\theta}{2} (\sigma_j)}$.
- 15) $R_{j}(\theta)\otimes I_2= e^{\pm i\frac{\theta}{2}(\sigma_j \otimes I_2)}$.
- 16) $R_{j}(-\theta)=R_{j}(\theta)^{\dagger}$.


- 17) $UR_x(\theta)U^{\dagger}=e^{i\frac{\theta}{2}(UXU^{\dagger})}$ (Conjugation by Unitary).
- 18) $(H\otimes H) CNOT_{(0,1)}(H\otimes H) = CNOT_{(1,0)}$ (Conjugation by Hadamard a.k.a Phase kickback).
- 19) $(I_2 \otimes S)e^{i\frac{\theta}{2} (X\otimes X)}(I_2 \otimes S^{\dagger})=e^{i\frac{\theta}{2} (X\otimes Y)}$ (Conjugation by $S$).


- 20) $ CNOT_{(j,k)}(X\otimes I_2) CNOT_{(j,k)} = X \otimes X$ (Conjugation by CNOT).

Plugging (15) into (17) and using (20):
- 21) $CNOT_{(j,k)} (R_x(\theta)\otimes I_2)CNOT_{(j,k)}=e^{i\frac{\theta}{2} CNOT_{(j,k)}(X\otimes I_2)CNOT_{(j,k)}}=e^{i\frac{\theta}{2}(X\otimes X)}$.

Identity 21 represents the transpilation procedure that converts a 2-qubit gate into a 2-qubit circuit. It can be extended as follows:

\begin{equation}
R_{xx}(\theta)=e^{\pm i\frac{\theta}{2}(X\otimes X)}=cos\left(\frac{\theta}{2}\right)(I_2\otimes I_2)\pm sin\left(\frac{\theta}{2}\right)(X\otimes X),\\
R_{yy}(\theta)= e^{\pm i\frac{\theta}{2}(Y\otimes Y)}=cos\left(\frac{\theta}{2}\right)(I_2\otimes I_2)\pm sin\left(\frac{\theta}{2}\right)(Y\otimes Y),\\
R_{zz}(\theta) = e^{\pm i\frac{\theta}{2}(Z\otimes Z)}=cos\left(\frac{\theta}{2}\right)(I_2\otimes I_2)\pm sin\left(\frac{\theta}{2}\right)(Z\otimes Z).
\end{equation}

## Gate Decomposition

### Single qubit gate decomposition:

- A Unitary gate U acting on a single qubit can always be decomposed into (See Ref. [1] Corollary 4.2 on page 176, or [Barenco et al.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.52.3457) Lemma 4.3):

\begin{equation}
U = e^{i\alpha}AXBXC,
\end{equation}

where $\alpha$ is a overall phase factor and $ABC=I$ for unitary operators $A$, $B$, and $C \in SU(2)$ acting on a single qubit.

- Z-Y-Z Decomposition for a single qubit unitary gate U (See [Barenco et al.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.52.3457)):

\begin{equation}
U = e^{i\alpha}R_z(\beta)R_y(\gamma)R_z(\delta),
\end{equation}

where $\alpha$, $\beta$, $\gamma$ and $\delta$ are real-valued numbers.

- X-Y-X Decomposition obtained via similarity transformation:

\begin{eqnarray}
U &=& R_x(\beta)R_y(\gamma)R_x(\delta)\\
\implies V &=& CUC^{\dagger} = R_z(\beta)R_y(\gamma)R_z(\delta),
\end{eqnarray}

- V-Z Decomposition:

\begin{equation}
U = e^{i\alpha}R_z(\beta)V^{\dagger}R_z(\gamma)VR_z(\delta).
\end{equation}

Where:

- $V=X^{1/2}$ is the square root of $X$.
- $C = S^{\dagger}$.
- $R_z(\theta)=V^{\dagger}R_y(\theta)V$.

### Two-qubit gate decomposition:

A Control-U gate writes:

\begin{equation}
C(U) = \Phi_1(\theta)A_2C(X)B_2C(X)C_2,
\end{equation}
where
\begin{equation}
\Phi_1 = \begin{pmatrix} 1 & 0 \\ 0 & e^{i\theta} \end{pmatrix} \otimes I_{2x2},
\end{equation}

is a phase gate applied to the first qubit and $A_2$, $B_2$ and $C_2$ are gates applied to the second qubit.

## Gate implementation with NumPy and Qiskit  

- $U3(\theta, \phi, \lambda)$

In [20]:
'''
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library.standard_gates import U3Gate
from qiskit.circuit import Parameter
'''

theta=Parameter('theta') 
phi=Parameter('phi') 
lam=Parameter('lam') 

qr=QuantumRegister(1)
qc = QuantumCircuit(qr)

gate=U3Gate(theta, phi, lam, label=None) # Uncomment this line if you need variables as placeholders.
#gate=U3Gate(np.pi, 0, 0, label=None) # Comment this line if you need variables as placeholders.
qc.append(gate,qr)
qc.draw()

┌───────────────────┐
q0: ┤ U3(theta,phi,lam) ├
    └───────────────────┘

- $CX = I\otimes |0\rangle\langle 0\rangle + X \otimes |1 \rangle \langle 1|$

In [21]:
# Qiskit Implementation of CX (CNOT) gate.

'''
import qiskit.quantum_info as qi
'''

qr=QuantumRegister(2)
qc = QuantumCircuit(qr)
qc.cnot(0,1)

print(qc.draw())
qi.Operator(qc)

           
q1_0: ──■──
      ┌─┴─┐
q1_1: ┤ X ├
      └───┘


Operator([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
          [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))

In [22]:
# NumPy Implementation of CX (CNOT) gate.

'''
import numpy as np
'''

psi_0 = np.array([1, 0])
psi_1 = np.array([0, 1])

outer0 = np.outer(psi_0, psi_0)
outer1 = np.outer(psi_1, psi_1)

id=np.identity(2)

X=np.array([[0, 1], [1, 0]]) 

cx = np.kron(id, outer0) + np.kron(X, outer1)
cx

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]])

- $CCX = I\otimes I\otimes |0\rangle\langle 0\rangle + CX \otimes |1 \rangle \langle 1|$

In [23]:
# Qiskit Implementation of CCX gate a.k.a Toffoli gate.

'''
from qiskit import QuantumCircuit, QuantumRegister,transpile
from qiskit import BasicAer
'''

backend = BasicAer.get_backend('unitary_simulator')
q = QuantumRegister(3)
qc = QuantumCircuit(q)
qc.ccx(q[0], q[1], q[2])

print(qc.draw())
job = backend.run(transpile(qc, backend))
job.result().get_unitary(qc, decimals=3)

           
q2_0: ──■──
        │  
q2_1: ──■──
      ┌─┴─┐
q2_2: ┤ X ├
      └───┘


array([[1.-0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.-0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.-0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.-0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.-0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.-0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.-0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.-0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [24]:
# NumPy Implementation of CCX gate a.k.a Toffoli gate.

id4 = np.kron(id, id)
ccx = np.kron(id4, outer0) + np.kron(cx, outer1)
ccx

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.]])

- $CCCX = I\otimes I\otimes I\otimes |0\rangle\langle 0\rangle + CCX \otimes |1 \rangle \langle 1|$

In [25]:
# NumPy Implementation of CCCX gate.

cccx = np.kron(np.identity(8), outer0) + np.kron(ccx, outer1)
cccx

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

- $CCRY = I\otimes I\otimes |0\rangle\langle 0\rangle + CRY \otimes |1 \rangle \langle 1|$

In [26]:
# Qiskit Implementation of CCRY gate.

'''
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library.standard_gates import RYGate
import qiskit.quantum_info as qi
'''

theta=np.pi
qr=QuantumRegister(3)
qc = QuantumCircuit(qr)
control=RYGate(theta).control(2)
qc.append(control,qr)

print(qc.draw())
qi.Operator(qc)

                
q12_0: ────■────
           │    
q12_1: ────■────
       ┌───┴───┐
q12_2: ┤ Ry(π) ├
       └───────┘


Operator([[ 1.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j],
          [ 0.00000000e+00+0.j,  1.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j],
          [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  1.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j],
          [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            2.22044605e-16+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j, -1.00000000e+00+0.j],
          [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  1.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e

- $CCCRY = I\otimes I\otimes I\otimes |0\rangle\langle 0\rangle + CCRY \otimes |1 \rangle \langle 1|$ (16x16 matrix)

In [27]:
# Qiskit Implementation of CCCRY gate.

'''
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library.standard_gates import RYGate
import qiskit.quantum_info as qi
'''

theta=np.pi
qr=QuantumRegister(4)
qc = QuantumCircuit(qr)
control=RYGate(theta).control(3)
qc.append(control,qr)

print(qc.draw())
qi.Operator(qc)

                
q13_0: ────■────
           │    
q13_1: ────■────
           │    
q13_2: ────■────
       ┌───┴───┐
q13_3: ┤ Ry(π) ├
       └───────┘


Operator([[ 1.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j],
          [ 0.00000000e+00+0.j,  1.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j],
          [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  1.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j,  0.00000000e+0

# The Heisenberg XXX Spin-1/2 Lattice Model for $N=3$ Three Particles<a name="model" />  


The [Quantum Heisenberg XXX spin-1/2 model](https://en.wikipedia.org/wiki/Quantum_Heisenberg_model#XXX_model) is a particular case of the general [XYZ Quantum Heisenberg Model](https://en.wikipedia.org/wiki/Quantum_Heisenberg_model) when the coupling coefficients are $J_x = J_y = J_z \doteq J$. The particle interaction of the aforementioned spin model for a system of $N$ quantum spin-1/2 particles arranged in a 1D (one-dimensional) Markov chain, where each particle is represented by a qubit, is given by the following `Hamiltonian operator`:

\begin{eqnarray}
h_{\text{heis}} &=& \sum_{\langle jk \rangle =0}^{N-1} J \left(\sigma_x^{(j)}\sigma_x^{(k)} + \sigma_y^{(j)}\sigma_y^{(k)} + \sigma_z^{(j)}\sigma_z^{(k)}\right).
\end{eqnarray}
Where: 
- $J$ is a real-valued [coupling](https://en.m.wikipedia.org/wiki/Coupling_constant) constant that determines the interaction strength.
- $\langle jk \rangle$ specifies interaction only between nearest neighbor qubits $j$ and $k$ ($j,k \in \{0,1,2\}$).
- $\otimes$ denotes the [tensor product](https://en.wikipedia.org/wiki/Tensor_product#Tensor_product_of_linear_maps) symbol.

<br></br>
In classical mechanics, the Hamiltonian $H$ corresponds to the sum of kinetic and potential energy of a **time-independent holonomic and monogenic system** \[[Goldstein, H.](#Ref)]). In quantum mechanics, the Hamiltonian is promoted to a so-called Hermitian operator $\hat{H}$ represented by a self-adjoint complex matrix within the matrix mechanics formalism.

The Pauli operators in this 1D spin lattice satisfy the following Lie algebra for the (anti-)commutation relations:


\begin{eqnarray}
\text{commutator: } \{\sigma_a^{j}, \sigma_b^{k} \} = 2\delta_{jk}\delta_{ab}\sigma_0^{j},\\
\end{eqnarray}
\begin{eqnarray}
\text{anti-commutator: }
[\sigma_a^{j}, \sigma_b^{k}] = 2i\delta_{jk}\epsilon_{abc}\sigma_c^{j},
\end{eqnarray}

Where:

- $[\hat{A},\hat{B}]$ denotes the bilinear skew-symmetric map $[,]: \mathcal{g} x \mathcal{g} \rightarrow \mathcal{g}$ (a.k.a commutator) between operators $\hat{A}$ and $\hat{B}$:

\begin{align}
[\hat{A},\hat{B}]\doteq\hat{A}\hat{B} - \hat{B}\hat{A}.
\end{align}
- $i=\sqrt{-1}$ denotes the imaginary unit.
- $\epsilon_{abc}$ denotes the normalized Levi-Civita symbol ($\epsilon_{xyz}=1$) with $a, b, c \in \{x,y,z\}$:

\begin{align*}
\varepsilon_{abc} \doteq
\begin{cases}
+1 & \text{for even permutation of }(a,b,c). \\
-1 & \text{for odd permutation of } (a,b,c).  \\
\;\;\,0 & \text{for any repeated index}.
\end{cases}
\end{align*}

- $\delta_{jk}$ denotes the Kronecker delta:

\begin{eqnarray}
\delta_{jk} \doteq \begin{cases}
0, & \mbox{if } j \ne k, \\
1, & \mbox{if } j=k. \end{cases}
\end{eqnarray}
- $\{\sigma_x,\sigma_y,\sigma_z\}$ denotes the $SU(2)$ set of [Pauli operators](https://en.wikipedia.org/wiki/Pauli_matrices) (2x2 complex Hermitian and Unitary matrices sharing equal eigenvalues of $\pm 1$).

For the particular case of a system of $N=3$ spin-1/2 particles arranged in a line, and setting $J=1$, one then has:

\begin{eqnarray}
h_{\text{heis}} &=& \sigma_x^{(0)}\otimes\sigma_x^{(1)}\otimes I^{(2)} + I^{(0)} \otimes\sigma_x^{(1)}\otimes\sigma_x^{(2)} + \sigma_y^{(0)}\otimes\sigma_y^{(1)}\otimes I^{(2)} + I^{(0)} \otimes \sigma_y^{(1)}\otimes\sigma_y^{(2)} + I^{(0)} \otimes\sigma_z^{(0)}\otimes\sigma_z^{(1)} + I^{(0)}\otimes\sigma_z^{(1)}\otimes\sigma_z^{(2)}.
\end{eqnarray}

However, the convention adopted by the quantum community is the following shorthand notation (omitting the tensor product ($\otimes$) and the 2x2 identity matrix $I$):

\begin{eqnarray}
h_{\text{heis3}} = \sigma_x^{(0)}\sigma_x^{(1)} + \sigma_x^{(1)}\sigma_x^{(2)} + \sigma_y^{(0)}\sigma_y^{(1)} + \sigma_y^{(1)}\sigma_y^{(2)} + \sigma_z^{(0)}\sigma_z^{(1)} + \sigma_z^{(1)}\sigma_z^{(2)}.
\end{eqnarray}

# Time Evolution of the initial state $|110\rangle$ under $h_{heis3}$ according to $U_{\text{heis3}}(t=\pi)$<a name="evolution" />  


To see how the state of the quantum system envolves from the initial state $|110\rangle$ to the final state $U_{\text{Heis3}}(t=\pi)|110\rangle = |\psi(\pi)\rangle$ with probability $|\langle 110 | U_{\text{Heis3}}(\pi) |110\rangle|^2$, one should recall the first and second postulates of quantum mechanics in the ubiquitous Dirac bra-ket notation.

## **Probability Over Time $t$**

Consider a physical observable, which is always represented by a Hermitian ($\hat{\mathcal{O}}=\hat{\mathcal{O}}^{\dagger}$) and, therefore, Normal ($ \hat{\mathcal{O}}\hat{\mathcal{O}}^{\dagger}=\hat{\mathcal{O}}^{\dagger}\hat{\mathcal{O}}$) operator with real eigenvalues $o_j$. A diagonal representation (See the spectral theorem) can be given by:
\begin{eqnarray}
\hat{\mathcal{O}} = \sum_{j=1}^d o_j P_{o_j}= \sum_{j=1}^d o_j |o_j\rangle \langle o_j|.
\end{eqnarray}

Where $P_{o_j}=|o_j\rangle \langle o_j|$ is the projector onto the eigenspace of the observable $\hat{\mathcal{O}}$ in some $d$-dimensional orthonormal basis $\{|o_j \rangle\}|_{j=1}^d$ with an orthonormal set of eigenvectors $|o_j \rangle$ ($\langle o_j|o_k\rangle=\delta_{jk}$).

If the quantum system is prepared in the state $|\psi\rangle \doteq \sum_{j=1}^d c_j|o_j\rangle$, a projective measurement (a.k.a von Neumann measurement) entails the following conditional probability for finding the system in the eigenstate $|o_j\rangle$ with eigenvalue $o_j$:

\begin{eqnarray} 
Pr(o_j|\psi\rangle)&=&\langle \psi | P^{\dagger}_{o_j} P_{o_j} |\psi \rangle \\ 
&=& \langle \psi | P_{o_j}^2 |\psi \rangle\\
&=& \langle \psi| (|o_j\rangle \langle o_j|o_j\rangle \langle o_j|)| \psi \rangle\\
&=&\delta_{jj}\langle\psi|o_j\rangle \langle o_j |\psi\rangle \\ 
&=& |\langle \psi|o_j \rangle|^2=|\langle o_j|\psi \rangle|^2\\
&=& \left|\langle o_j|\sum_{k=1}^d c_k |o_k\rangle\right|^2\\
&=&  \left|\sum_{k=1}^d c_k \langle o_j|o_k\rangle\right|^2\\
&=&\left|\sum_{k=1}^d c_k \delta_{jk} \right|^2 = |c_j|^2.
\end{eqnarray} 

This result is also known as Born's rule:

\begin{eqnarray}
Pr(|o_j\rangle) \doteq |\langle o_j |\psi \rangle|^2 = |c_j|^2.
\end{eqnarray}

<br></br>
If the system has evolved from the initial state $|\psi(0)\rangle$ to the final state $|\psi(t)\rangle$, then:

\begin{eqnarray}
P_t(|o_j\rangle) &=& |\langle o_j | \hat{U}(t) |\psi(0)\rangle|^2 \\
&=& |\langle o_j |\psi(t)\rangle|^2.
\end{eqnarray}

Therefore, the probability of measuring the initial state $|\psi(0)\rangle=|110\rangle$ over a time window $t=\pi$ under the **time-independent Hamiltonian** XXX Heisemberg Hamiltonian is:

\begin{eqnarray}
P_{t=\pi}(|110\rangle) &=& |\langle 110 | u_{heis3}(t=\pi) |110\rangle|^2 \\
&=& |\langle 110 |e^{-ih_{heis3}\pi / \hbar} |110\rangle|^2.
\end{eqnarray}

Q.E.D.

This last equation is known as the transition probability. It resembles the modulus squared of the expectation value of an observable $\hat{O}$ as in $|\langle \hat{O}\rangle|^2$. **However, the evolution operator is not an Hermitian operator and, therefore, it does not describe a physical observable.** Expected value only makes sense for physical observables!

The expectation value of a physical observable $\hat{\mathcal{O}}$ is:

\begin{align}
\langle \hat{\mathcal{O}}\rangle &=\sum_{j=1}^d o_j Pr(o_j|\psi\rangle) \\
&=\sum_{j=1}^d o_j  \langle \psi |o_j \rangle \langle o_j|\psi \rangle \\
&= \langle \psi | \left(\sum_{j=1}^d o_j |o_j \rangle \langle o_j| \right) |\psi \rangle\\
&= \langle \psi | \left(\sum_{j=1}^d o_j P_{o_j} \right) |\psi \rangle\\
&=\langle \psi |\hat{\mathcal{O}}|\psi\rangle.
\end{align}

Upon evolution of the system's state vector ($\hat{U}(t)|\psi\rangle$), the expected value becomes:

\begin{align}
\langle \psi |U^{\dagger}\hat{\mathcal{O}}\hat{U}|\psi\rangle.
\end{align}

# The Trotter-Susuki formula<a name="trotter" />  

Consider the following decomposition for a general Hamiltonian $\hat{H}$:

\begin{align}
\hat{H} = \hat{H}_a + \hat{H}_b.
\end{align}

If the reduced Hamiltonians commute, i.e, $[\hat{H}_{a},\hat{H}_{b}]=\mathbb{O}$, then the evolution operator is a product of reduced evolution operators:

\begin{align}
\hat{U} = e^{-i\hat{H}t/ħ} = e^{-i\hat{H}_{a}t/\hbar-i\hat{H}_{b}t/\hbar} = e^{-i\hat{H}_{a}t/\hbar}e^{-i\hat{H}_{b}t/\hbar} = \hat{U}_{a}\hat{U}_{b} = \hat{U}_{b}\hat{U}_{a}.
\end{align}






**Simultaneous diagonalization theorem** *Suppose $A$ and $B$ are two Hermitian operators. Then $[A, B] = AB-BA =\mathbb{O}$ is equal to the zero matrix (they commute) if, and only if, there exists an orthonormal basis such that both $A$ and $B$ are diagonal with respect to that basis, i.e, they share the same set of eigenvectors. Therefore, $A$ and $B$ are simultaneously diagonalizable with different eigenvalues.*

In the case where $[H_a^{(0,1)},H_b^{(1,2)}]\ne\mathbb{O}$, one cannot simply write the evolution operator $\hat{U}$ as a product of evolution operators, therefore, another method must be adopted. There are several techniques to approximate a time Unitary evolution $\hat{U}=e^{-i\hat{H}t/\hbar}$ into gate operations for quantum simulation, to name a few:

1. Trotter-Susuki formula.
2. Randomized evolution (Qdrift, density matrix exponentiation).
3. Linear combination of unitaries.
4. Quantum Walks (Qubitization).



The Trotter-Susuki formula is defined as (setting $\hbar\equiv1$):

\begin{align}
U_{Trotter}(t) = e^{-it\Big(\hat{H}_a + \hat{H}_b\Big)} \equiv \lim_{N\rightarrow\infty}\Big(e^{-it \hat{H}_a/N}e^{-it \hat{H}_b /N}\Big)^{N}.
\end{align}

Advantages of using Trotterization:

- The algorithm is ancilla-free.

- It leverages the commutative property of the component Hamiltonians to achieve efficient simulation.

- It preserves the locality property of some unidimensional systems with nearest neighbor interactions enabling quantum simulation speed up.







The outline of the Trotterization algorithm a.k.a as product-formula method or splitting method is as follows:

**1.** Write the Hamiltonian operator $\hat{H}$ of the system of interest in the form:

\begin{eqnarray}
\hat{H}= \sum_{\gamma=1}^{\Gamma} \hat{H}_{\gamma},
\end{eqnarray}
where $\hat{H}_{\gamma}$ is itself a Hermitian operator.

**2.** Write the unitary evolution operator $U$ in terms of product of exponentials using the first-order Lie-Trotter formula defined as:

\begin{eqnarray}
\mathcal{L}_1(t) = e^{-it \hat{H}_{1}} \cdots e^{-it \hat{H}_{\Gamma}} = e^{-it \hat{H}} + O(t^2),
\end{eqnarray}
where $O(t^2)$ is the trotter error.

**3.** Perform $e^{-it\hat{H}}$ up to some error $\epsilon$ according to the spectral norm:

\begin{eqnarray}
||\hat{U}-e^{-it\hat{H}}|| \leq \epsilon.
\end{eqnarray}

**4.** Define the total cost of the simulation as:
\begin{eqnarray}
\text{total cost} = \text{no. of steps } \times \text{ cost}/\text{step}.
\end{eqnarray}

In step 3, each exponential in the evolution operator $U_{\text{Heis3}}(t)$ is transpiled into a quantum gate.

# Decomposition of $U_{\text{Heis3}}(t)$ using Trotterization<a name="decomp" />  

Consider the following decomposition for the Hamiltonian $H_{heis3}$:

\begin{align}
H_{heis3} = H_a^{(0,1)} + H_b^{(1,2)},
\end{align}
where
\begin{align}
H_a^{(0,1)}  =\big(X^{(0)} \otimes X^{(1)} + Y^{(0)}\otimes Y^{(1)} + Z^{(0)}\otimes Z^{(1)}\big)\otimes I^{(2)}, \\
H_b^{(1,2)} =I^{(0)}\otimes \big(X^{(1)}\otimes X^{(2)} +  Y^{(1)}\otimes Y^{(2)} +  Z^{(1)}\otimes Z^{(2)}\big),
\end{align}

We note that the pair of operators in the exponential of $U_{Trotter}(t)$ commute, i.e,

$$[X\otimes X,Y\otimes Y] = [X\otimes X,Z\otimes Z] = [Y\otimes Y,Z\otimes Z] =\mathbb{O}.$$

One can verify this is true, as follows ($j\ne k$):

\begin{eqnarray}
[\sigma_{j}\otimes\sigma_{j},\sigma_{k}\otimes\sigma_{k}] & = (\sigma_{j}\otimes\sigma_{j}) \cdot (\sigma_{k}\otimes\sigma_{k}) - (\sigma_{k}\otimes\sigma_{k}) \cdot (\sigma_{j}\otimes\sigma_{j}),
\end{eqnarray}

using $(A\otimes B)\cdot(C\otimes D)=AC\otimes BD$, one then has

\begin{eqnarray}
[\sigma_{j}\otimes\sigma_{j},\sigma_{k}\otimes\sigma_{k}]=\sigma_{j}\sigma_{k}\otimes\sigma_{j}\sigma_{k} - \sigma_{k}\sigma_{j}\otimes\sigma_{k}\sigma_{j} 
\end{eqnarray}

and given the $\mathfrak{su}(2)$ Lie algebra $\sigma_{j}\sigma_{k} = \sigma_{0}\delta_{jk}+i\epsilon_{jkl}\sigma_{l}$, it becomes

\begin{eqnarray}
[\sigma_{j}\otimes\sigma_{j},\sigma_{k}\otimes\sigma_{k}]&=& i\epsilon_{jkl}\sigma_{l}\otimes i\epsilon_{jkl}\sigma_{l} -i\epsilon_{kjl}\sigma_{l}\otimes i\epsilon_{kjl}\sigma_{l} \\
&=& i^{2}\epsilon_{jkl}^{2}\sigma_{l}\otimes\sigma_{l} -i^{2}\epsilon_{kjl}^{2}\sigma_{l}\otimes \sigma_{l} \\
&=& i^{2}\epsilon_{jkl}^{2}\sigma_{l}\otimes\sigma_{l} -i^{2}(-\epsilon_{jkl})^{2}\sigma_{l}\otimes \sigma_{l} \\
&=& \mathbb{O}.
\end{eqnarray}


With the above result in hands, the exponential $e^{-itH_a^{01}} $ can be written in the form:

\begin{eqnarray}
e^{-itH_a^{01}} &=& e^{-it(X\otimes X+ Y\otimes Y + Z\otimes Z)\otimes I} \\ 
&=& e^{-it(X\otimes X\otimes I + Y\otimes Y\otimes I + Z\otimes Z\otimes I)} \\
&=& e^{-itX\otimes X\otimes I}e^{-itY\otimes Y\otimes I}e^{-itZ\otimes Z\otimes I},
\end{eqnarray}

which after using $e^{A\otimes I} = e^{A}\otimes I$, one gets

\begin{eqnarray}
e^{-itH_a^{01}}&=& \Big(e^{-itX\otimes X}\otimes I\Big)\Big(e^{-itY\otimes Y}\otimes I\Big)\Big(e^{-itZ\otimes Z}\otimes I\Big), \\
&=& \Big(e^{-itX\otimes X}\Big)\Big(e^{-itY\otimes Y}\Big)\Big(e^{-itZ\otimes Z}\Big)\otimes I.
\end{eqnarray}

And equivalenty for $e^{-itH_b^{12}}$:

\begin{eqnarray}
e^{-itH_b^{12}} &= I\otimes\Big( e^{-itX\otimes X}\Big)\Big( e^{-itY\otimes Y}\Big)\Big( e^{-itZ\otimes Z}\Big).
\end{eqnarray}

The unitary evolution operator for the decomposition of $H_{heis3}$ according to the Trotter-Suzuki fomula thus becomes:

\begin{eqnarray}
U_{\text{Heis3}}(t) &=& e^{-itH_{heis3}} = e^{-it\Big(H_a^{(0,1)} + H_b^{(1,2)}\Big)} =  \lim_{N\rightarrow\infty}\Big(e^{-it H_a^{(0,1)}/N}e^{-it H_b^{(1,2)} /N}\Big)^{N}\\
&=& \lim_{N\rightarrow\infty}\left(\Big(e^{-it X^0\otimes X^1/N}e^{-it Y^0\otimes Y^1/N}e^{-it Z^0\otimes Z^1/N}\otimes I\Big)\Big(I\otimes e^{-it X^1\otimes X^2/N}e^{-it Y^1\otimes Y^2/N}e^{-it Z^1 \otimes Z^2/N}\Big)\right)^{N}.
\end{eqnarray}

Making the definitions

\begin{eqnarray}
XX(2t)^{(j,k)}\equiv e^{-it X^j \otimes X^k}, \\
YY(2t)^{(j,k)}\equiv e^{-it Y^j \otimes Y^k}, \\
ZZ(2t)^{(j,k)}\equiv e^{-it Z^j \otimes Z^k},
\end{eqnarray}

one then has:

\begin{eqnarray}
U_{\text{Heis3}}(t) = \lim_{N\rightarrow\infty}\Big[\Big(XX(2t/N)^{(0,1)}YY(2t/N)^{(0,1)}ZZ(2t/N)^{(0,1)}\otimes I\Big)\Big(I\otimes XX(2t/N)^{(1,2)}YY(2t/N)^{(1,2)}ZZ(2t/N)^{(1,2)}\Big)\Big]^{N}.
\end{eqnarray}

# Transpilation of $U_{\text{Heis3}}(t)$ Into Quantum Gates<a name="transp" />  

To build quantum circuits corresponding to unitary evolution operators from exponentials of the form $U=e^{-i\hat{H}\Delta t}$, one can use the parity trick (see Ref. [1], Sec. 4.7.3). In the next sections, the provided quantum circuits are verified.

For a Hamiltonian of the form $\hat{H}=\sum_j^N \hat{H}_j = \sum_j^N \lambda P_j \otimes P_j$, with Pauli operators $P_j \in \{X, Y, Z\}$ and constant $\lambda$, transpilation into quantum gates of each one of the components can be achieved with the following steps:

**1) Diagonalize the component Hamiltonian $\hat{H}_j=\lambda P_j \otimes P_j$ in the computational basis:**

$$ \lambda (R_P\otimes R_P) (Z\otimes Z) (R_P\otimes R_P),$$ 

where $R_P \in \{R_x, R_y, R_z \}$ is a local rotation (one-qubit gate) around the $P$-axis.

**2) Compute the parity information (phase shift) in the basis $|ij\rangle$ (usually implemented with CNOT gates):**

$$(P \otimes P)|ij\rangle = (-1)^{i \oplus j }|ij\rangle,$$ 

where $\oplus$ denotes the exclusive-OR symbol that represents the "direct sum" in mathematics and the parity in quantum mechanics.

**3) Compute the gate that kicks back the right parity (phase) information:**

$$e^{-i\lambda(-1)^{i\oplus j}\: Zt}.$$

**4) Uncompute the parity information. This is done by applying the Hermitian conjugate of the gate used to compute the parity.**

## The $e^{-itZZ}$ gate

The $e^{-itZZ}$ gate (a.k.a [RZZ](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RZZGate.html)) has the following representation:

\begin{equation}
R_{zz}(2t)= \exp(-it Z\otimes Z)=cos\left(t\right)(I_2\otimes I_2)-sin\left(t\right)(Z\otimes Z).
\end{equation}

IBM supplementary material provides the following quantum circuit for the $e^{-itZZ}$ gate:

In [28]:
# e^{(-itZZ)}

'''
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
'''

t = Parameter('t')
qc = QuantumCircuit(2)
qc.cnot(0,1)
qc.rz(2 * t, 1)
qc.cnot(0,1)
#qc.draw(output='mpl') # Requires 'pip install pylatexenc' library to use 'MatplotlibDrawer'.
qc.draw()

q_0: ──■───────────────■──
     ┌─┴─┐┌─────────┐┌─┴─┐
q_1: ┤ X ├┤ Rz(2*t) ├┤ X ├
     └───┘└─────────┘└───┘

The above circuit can be obtained using the parity trick, where the first CNOT gate computes the parity, and the second CNOT gate uncomputes the operation, since the CNOT gate is an Hermitian operator ($\text{CNOT} \cdot \text{CNOT}^{\dagger} = I$).

**Verification:**

**1)** This gate is already diagonalized in the Z-basis. 

**2)** One should check the parity information by computing the action of the $ZZ$ operator:

\begin{eqnarray}
ZZ|00\rangle &=& |00\rangle, \\
ZZ|10\rangle &=& -|10\rangle, \\
ZZ|01\rangle &=& -|01\rangle, \\
ZZ|11\rangle &=& |11\rangle. \\
\end{eqnarray}

One can see that a minus sign (phase) is introduced whenever the basis state has odd parity (01 or 10). Therefore, a phase shift will be applied to the system according to $e^{i\Delta t}$ if the parity of the $n$ qubits in the computational basis is odd, otherwise, the phase shift will be $e^{-i\Delta t}$ if the parity is even (see Ref. [1], Sec. 4.7.3).

**3)** One should recall that $RZ(2t) = e^{-itZ}$ (see Section "Gate Algebra and Circuit Implementation" of this supplementary material). 

**Implementing CNOTs and the $RZ(2t)$ gate for the parity computation circuit from steps 2) to 4):**

\begin{eqnarray}
CX^{01}\big(I\otimes RZ(2t)\big)CX^{01} =
CX^{01}\big(I\otimes e^{-itZ}\big)CX^{01}\\ 
\end{eqnarray}

From the aforementioned chapter, using the equation for a generic CU gate one can write $CX^{01} = I\otimes |0\rangle\langle 0\rangle + X \otimes |1 \rangle \langle 1|$. However, in textbook convention this gate reads $CX^{01}=|0\rangle\langle 0|\otimes I +|1\rangle\langle 1|\otimes X$. One then has:

\begin{eqnarray}
CX^{01}\big(I\otimes e^{-itZ}\big)CX^{01}=(|0\rangle\langle 0|\otimes I +|1\rangle\langle 1|\otimes X)\big(I\otimes e^{-itZ}\big)\big(|0\rangle\langle 0|\otimes I +|1\rangle\langle 1|\otimes X \big).
\end{eqnarray}

From here, one should note that since each term between a tensor product is a matrix, the distributivity property for matrix product with respect to matrix addition yelds:

$$\big(I\otimes e^{-itZ}\big)\big(|0\rangle\langle 0|\otimes I +|1\rangle\langle 1|\otimes X \big) = \big(I\otimes e^{-itZ}\big) \big(|0\rangle\langle 0|\otimes I\big) + \big(I\otimes e^{-itZ}\big)\big(|1\rangle\langle 1|\otimes X \big),$$

Using $(A\otimes B)\cdot(C\otimes D)=A\cdot C\otimes B\cdot D$, the last term becomes:

$$\big(I \cdot |0\rangle\langle 0|\otimes e^{-itZ} + I \cdot |1\rangle\langle 1|\otimes e^{-itZ}X \big) = \big(|0\rangle\langle 0|\otimes e^{-itZ} + |1\rangle\langle 1|\otimes e^{-itZ}X \big) .$$ 

The final steps carry on as follows:

\begin{eqnarray}
CX^{01}\big(I\otimes e^{-itZ}\big)CX^{01} &=& \big(|0\rangle\langle 0|\otimes I +|1\rangle\langle 1|\otimes X \big)\big(|0\rangle\langle 0|\otimes e^{-itZ} +|1\rangle\langle 1|\otimes e^{-itZ}X \big) \\
&=& |0\rangle\langle 0|\otimes e^{-itZ} +|1\rangle\langle 1|\otimes Xe^{-itZ}X \\ 
&=& |0\rangle\langle 0|\otimes \big(e^{-it}|0\rangle\langle 0|+e^{it}|1\rangle\langle 1|\big) +|1\rangle\langle 1|\otimes X \big(e^{-it}|0\rangle\langle 0|+e^{it}|1\rangle\langle 1|\big)X,
\end{eqnarray}

and noting the operation $X|0\rangle = |1\rangle$, one finally gets:

\begin{eqnarray}
CX^{01}\big(I\otimes e^{-itZ}\big)CX^{01}&=& |0\rangle\langle 0|\otimes \big(e^{-it}|0\rangle\langle 0|+e^{it}|1\rangle\langle 1|\big) +|1\rangle\langle 1|\otimes \big(e^{-it}|1\rangle\langle 1|+e^{it}|0\rangle\langle 0|\big) \\
&=& e^{-it}|00\rangle\langle 00| + e^{it}|01\rangle\langle 01| + e^{-it}|11\rangle\langle 11| + e^{it}|10\rangle\langle 10| \\
&=& e^{-itZZ}.
\end{eqnarray}


## The $e^{-itXX}$ gate

IBM supplementary material provides the following quantum circuit for $e^{-itXX}$:

In [29]:
# e^{(-itXX)}

'''
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
'''

t = Parameter('t')
qc = QuantumCircuit(2)
qc.ry(np.pi/2,[0,1])
qc.cnot(0,1)
qc.rz(2 * t, 1)
qc.cnot(0,1)
qc.ry(-np.pi/2,[0,1])

qc.draw()

┌─────────┐                     ┌──────────┐
q_0: ┤ Ry(π/2) ├──■───────────────■──┤ Ry(-π/2) ├
     ├─────────┤┌─┴─┐┌─────────┐┌─┴─┐├──────────┤
q_1: ┤ Ry(π/2) ├┤ X ├┤ Rz(2*t) ├┤ X ├┤ Ry(-π/2) ├
     └─────────┘└───┘└─────────┘└───┘└──────────┘

The above circuit can be obtained using the parity trick and applying a change of basis between Pauli $X$ and $Z$ bases with the Hadamard gate. Since the exponential is not diagonalized in the computational basis (Z-basis), the first pair of Hadamard gates are used to do a basis change (rotation) to the $X$-basis, while the second pair of Hadamard gates change the basis from $X$ back to $Z$ because measurements are made in the Z-basis. The middle circuit then computes the parity.

From the fact that 

$$H|0\rangle = R_y(\pi/2)|0\rangle = |+\rangle = \frac{1}{\sqrt{2}}(|0\rangle+|1\rangle),$$ 

and 

$$H|1\rangle = R_y(-\pi/2)|0\rangle = |-\rangle = \frac{1}{\sqrt{2}}(|0\rangle-|1\rangle),$$ 

one equivalent quantum circuit is:

In [30]:
# Equivalent circuit:

qc = QuantumCircuit(2)
qc.h([0,1])
qc.cx(0, 1)
qc.rz(2 * t, 1)
qc.cx(0, 1)
qc.h([0,1])
qc.draw()

┌───┐                     ┌───┐
q_0: ┤ H ├──■───────────────■──┤ H ├
     ├───┤┌─┴─┐┌─────────┐┌─┴─┐├───┤
q_1: ┤ H ├┤ X ├┤ Rz(2*t) ├┤ X ├┤ H ├
     └───┘└───┘└─────────┘└───┘└───┘

**Verification:**



One should note that $e^{-itXX}$ is shorthand for $e^{-itX \otimes X}.$ Also, recall from the spectral decomposition theorem for Normal matrices that any Normal operator $\hat{O}$ in a Hilbert space $\mathcal{H}$ of dimension $\dim \mathcal{H} =d$ is unitarily diagonalizable and has a spectral (eigenvalue) decomposition in terms of the outer product representation, and in the basis of its eigenvectors, of the form:

$$ \hat{O} = \sum_{j=1}^{d}= o_j P_{o_j} = \sum_{j=1}^{d} o_j |o_j \rangle \langle o_j|. \tag{1} $$

Where:

- $o_j$ and $P_{o_j}$ are the corresponding eigenvalue and projector operator of the observable $\hat{O}$, respectively. 

- $\{o_j\}|_{j=1}^d$ is a basis set of $d$ linearly independent orthonormal eigenvectors $|o_j\rangle$ of $\hat{O}$ with eigenvalue $o_j$.

To see this is true, note that:

$$ \Bigg( \sum_{j=1}^{d} o_j |o_j \rangle \langle o_j| \Bigg) |o_k \rangle =  \sum_{j=1}^{d} o_j |o_j \rangle \delta_{jk}= o_k |o_k \rangle = \hat{O} |o_k \rangle.$$

Moving on... Since Pauli operators are all involutory operators, i.e, their eigenvalues are $o_j = \pm 1$, one then has 

$$X = \sum_{j=1}^{d} x_j |x_j \rangle \langle x_j| = |+\rangle\langle +|-|-\rangle\langle -|. \tag{2}$$

Where $$ |\pm\rangle \equiv \frac{1}{\sqrt{2}}(|0\rangle\pm|1\rangle) \tag{3} $$ 

denotes the two orthonormal eigenstates (eigenvectors) of the Pauli $X$-gate according to:

$$X |+\rangle = \frac{1}{\sqrt{2}}(X|0\rangle+X|1\rangle)=\frac{1}{\sqrt{2}}(|1\rangle+|0\rangle)=|+\rangle, \tag{4}$$
$$X |-\rangle = \frac{1}{\sqrt{2}}(X|0\rangle-X|1\rangle) =\frac{1}{\sqrt{2}}(|1\rangle-|0\rangle)=-|-\rangle. \tag{5}$$

With that, one has:

\begin{eqnarray}
XX &\equiv& X\otimes X = (|+\rangle\langle +|-|-\rangle\langle -|)\otimes(|+\rangle\langle +|-|-\rangle\langle -|) \tag{6}\\
&=& (|+\rangle\langle +|) \otimes (|+\rangle\langle +|) - (|+\rangle\langle +|) \otimes (|-\rangle\langle -|) - (|-\rangle\langle -|)\otimes (|+\rangle\langle +|) + (|-\rangle\langle -|) \otimes (|-\rangle\langle -|) \tag{7} \\
&=& (|++\rangle\langle ++|)+(|--\rangle\langle --|)-(|+-\rangle\langle +-|)+(|-+\rangle\langle -+|) \tag{8} \\
&\implies & e^{-itX \otimes X} = e^{-it}\bigg(|++\rangle\langle ++|+|--\rangle\langle --|\bigg)+e^{it}\bigg(|+-\rangle\langle +-|+|-+\rangle\langle -+|\bigg) \tag{9}.
\end{eqnarray}

- Eq. (7) was obtained from Eq. (6) using the distributive property:

$$(|a\rangle + |b\rangle)\otimes(|c\rangle + |d\rangle)=|a\rangle \otimes |c\rangle + |a\rangle \otimes |d\rangle + |b\rangle \otimes |c\rangle + |b\rangle \otimes |d\rangle. \tag{10}$$

- Eq. (8) was obtained from Eq. (7) using $(A\otimes B)(C\otimes D)=AC\otimes BD$, or equivalently:

$$|a\rangle \langle c| \otimes |b\rangle \langle d|  = (|a\rangle \otimes |b\rangle)(\langle c| \otimes \langle d|) \doteq |ab\rangle\langle cd|. \tag{11}$$

- Eq. (9) is left as an exercise (see the next cell for a motivation).

Applying a Hadamard gate to change between the $X$ and $Z$ bases ($H|0\rangle = R_y(\pi/2)|0\rangle = |+\rangle$ and $H|1\rangle = R_y(-\pi/2)|0\rangle = |-\rangle$), yields

\begin{eqnarray}
e^{-itX \otimes X} &=& e^{-it}\left[(H\otimes H)|00\rangle\langle 00|(H\otimes H)+(H\otimes H)|11\rangle\langle 11|(H\otimes H)\right]+e^{it}\left[(H\otimes H)|01\rangle\langle 01|(H\otimes H)+(H\otimes H)|10\rangle\langle 10|(H\otimes H)\right] \tag{12}\\
& =& (H\otimes H)\left[e^{-it}(|00\rangle\langle 00|+|11\rangle\langle 11|)+e^{it}(|01\rangle\langle 01|+|10\rangle\langle 10|)\right](H\otimes H) \tag{13}\\
& =& (H\otimes H)e^{-itZ\otimes Z}(H\otimes H) \tag{14}\\
& =& (H\otimes H)C_{x}^{01}\big(I\otimes e^{-itZ}\big)C_{x}^{01}(H\otimes H) \tag{15}\\
&  =& (H\otimes H)C_{x}^{01}\big(I\otimes RZ(2t)\big)C_{x}^{01}(H\otimes H). \tag{16}
\end{eqnarray}

**Note**: *some steps have been omitted. I encourage the reader to verify them.*

Specifically, show that:

\begin{eqnarray}
(|a\rangle + |b\rangle)\otimes(|c\rangle + |d\rangle)&=&|a\rangle \otimes |c\rangle + |a\rangle \otimes |d\rangle + |b\rangle \otimes |c\rangle + |b\rangle \otimes |d\rangle, \tag{17} \\
(|a\rangle \langle c|) \otimes (|b\rangle \langle d|)  &=& \big(|a\rangle \otimes |b\rangle\big)\big(\langle c| \otimes \langle d|\big), \tag{18}\\
|++\rangle\langle ++|&=&(H\otimes H)|00\rangle\langle 00|(H\otimes H), \tag{19}\\
|--\rangle\langle --|&=&(H\otimes H)|11\rangle\langle 11|(H\otimes H), \tag{20}\\
|+-\rangle\langle +-|&=&(H\otimes H)|01\rangle\langle 01|(H\otimes H), \tag{21}\\
|-+\rangle\langle -+|&=&(H\otimes H)|10\rangle\langle 10|(H\otimes H), \tag{22}\\
e^{-itZ\otimes Z} &=& C_{x}^{01}\big(I\otimes e^{-itZ}\big)C_{x}^{01}, \tag{23}\\
e^{-itZ} &=& RZ(2t) \tag{24}.
\end{eqnarray}

You can open a pool request to share your contribution. 


**Motivation:** To demonstrate Eq. (9), one should recall that if $\hat{O}$ is a Hermitian or Unitary operator then it is also a Normal operator and, therefore, has a spectral decomposition in terms of the outer product representation in the form $\hat{O} = \sum_j o_j|o_j\rangle \langle o_j|$ according to the spectral decomposition theorem for Normal matrices. In this case, it is possible to write the operator function (matrix function) on a normal matrix as:

$$f(\hat{O})= \sum_j f(o_j) |o_j\rangle \langle o_j|,$$

which is equivalent to (writing $\hat{O} = UDU^{\dagger}$)

\begin{eqnarray}
f(\hat{O}) = Uf(D)U^{\dagger}.
\end{eqnarray}


For the exponential function over the field of the complex numbers, it becomes:

$$ e^{i\theta\hat{O}} = \sum_{j=1}^n e^{i\theta o_j} |o_j\rangle \langle o_j| = e^{i\theta o_1} |o_{1}\rangle \langle o_{1}| + \cdots + e^{i\theta o_n} |o_{n}\rangle \langle o_{n}|. \tag{25}$$

To see this is true, recall that the eigenvalues of a matrix are multiplied by a scalar when the matrix is multiplied by the same scalar, while the eigenvectors are left unchanged. One can extend this identity to a tensor product of Normal operators noting that `the tensor product of two Hermitian operators is another Hermitian operator`. With that, one should be able to obtain Eq. (9).

## The $e^{-itYY}$ gate

IBM supplementary material provides the following quantum circuit for $e^{-itYY}$:

In [31]:
# e^{(-itZZ)}

'''
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
'''

t = Parameter('t')
qc = QuantumCircuit(2)
qc.rx(np.pi/2,[0,1])
qc.cnot(0,1)
qc.rz(2 * t, 1)
qc.cnot(0,1)
qc.rx(-np.pi/2,[0,1])

qc.draw()

┌─────────┐                     ┌──────────┐
q_0: ┤ Rx(π/2) ├──■───────────────■──┤ Rx(-π/2) ├
     ├─────────┤┌─┴─┐┌─────────┐┌─┴─┐├──────────┤
q_1: ┤ Rx(π/2) ├┤ X ├┤ Rz(2*t) ├┤ X ├┤ Rx(-π/2) ├
     └─────────┘└───┘└─────────┘└───┘└──────────┘

From the fact that $ SH|0\rangle = R_x(\pi/2)|0\rangle) = \frac{1}{\sqrt{2}}(|0\rangle+i|1\rangle) $, one equivalent quantum circuit is:

From the fact that 

$$SH|0\rangle = R_x(-\pi/2)|0\rangle = |\oplus\rangle = \frac{1}{\sqrt{2}}(|0\rangle+i|1\rangle),$$ 

and 

$$SH|1\rangle = R_x(\pi/2)|0\rangle = |\ominus\rangle = \frac{1}{\sqrt{2}}(|0\rangle-i|1\rangle),$$ 

one equivalent quantum circuit is:

In [32]:
# Equivalent circuit:

qc = QuantumCircuit(2)
qc.sdg([0,1])
qc.h([0,1])
qc.cx(0, 1)
qc.rz(2 * t, 1)
qc.cx(0, 1)
qc.h([0,1])
qc.s([0,1])
qc.draw()

┌─────┐┌───┐                     ┌───┐┌───┐
q_0: ┤ Sdg ├┤ H ├──■───────────────■──┤ H ├┤ S ├
     ├─────┤├───┤┌─┴─┐┌─────────┐┌─┴─┐├───┤├───┤
q_1: ┤ Sdg ├┤ H ├┤ X ├┤ Rz(2*t) ├┤ X ├┤ H ├┤ S ├
     └─────┘└───┘└───┘└─────────┘└───┘└───┘└───┘

**Verification:**


One should note that $e^{-itYY}$ is shorthand for $e^{-itY \otimes Y}$. Following the same thought process as before, one writes the Pauli operator $Y$ in the basis of its eigenvectors:

$$Y=|\oplus\rangle\langle \oplus|-|\ominus\rangle\langle \ominus|,$$

where

$$|+y\rangle \equiv |\oplus\rangle = \frac{1}{\sqrt{2}}(|0\rangle+i|1\rangle) \text{ and } |-y\rangle \equiv |\ominus\rangle = \frac{1}{\sqrt{2}}(|0\rangle-i|1\rangle).$$

Therefore,

\begin{eqnarray}
YY &\equiv& Y \otimes Y = (|\oplus\oplus\rangle\langle \oplus\oplus|+|\ominus\ominus\rangle\langle \ominus\ominus|)-(|\oplus\ominus\rangle\langle \oplus\ominus|+|\ominus\oplus\rangle\langle \ominus\oplus|) \\
&\implies& e^{-itY\otimes Y} = e^{-it}(|\oplus\oplus\rangle\langle \oplus\oplus|+|\ominus\ominus\rangle\langle \ominus\ominus|) + e^{it}(|\oplus\ominus\rangle\langle \oplus\ominus|+|\ominus\oplus\rangle\langle \ominus\oplus|).
\end{eqnarray}

The next step is to apply a basis change, as follows:

$$|\oplus\rangle = SH|0\rangle = R_x(-\pi/2)|0\rangle  \text{ and }  |\ominus\rangle = SH|1\rangle = R_x(\pi/2)|0\rangle,$$

such that

\begin{align}
e^{-itY\otimes Y} & = (S\otimes S)(H\otimes H)\Big(e^{-it}(|00\rangle\langle 00|+|11\rangle\langle 11|)+e^{it}(|01\rangle\langle 01|+|10\rangle\langle 10|)\Big)(H\otimes H)(S^{\dagger}\otimes S^{\dagger}) \\
& = (S\otimes S)(H\otimes H)e^{-itZ \otimes Z}(H\otimes H)(S^{\dagger}\otimes S^{\dagger}) \\
& = (S\otimes S)e^{-itX\otimes X}(S^{\dagger}\otimes S^{\dagger}) \\
& = (S\otimes S)(H\otimes H)C_{x}^{01}\big(I\otimes e^{-itZ}\big)C_{x}^{01}(H\otimes H)(S^{\dagger}\otimes S^{\dagger}) \\
& = (S\otimes S)(H\otimes H)C_{x}^{01}\big(I\otimes RZ(2t)\big)C_{x}^{01}(H\otimes H)(S^{\dagger}\otimes S^{\dagger}).
\end{align}

# State Fidelity<a name="fid" />  

In noisy environments, the evolved quantum state must be represented by a mixed density operator. The fidelity between two mixed density operators (density matrices) $\rho$ and $\sigma$ is:

\begin{align}
F(\rho,\sigma) = \left(tr \sqrt{\rho^{1/2} \sigma \rho^{1/2}}\right)^2,
\end{align}

with $\rho= p_j\sum_j|\psi_{j}\rangle \langle \psi_{j}|$ and $\sigma=p_k\sum_k|\psi_{k}\rangle \langle \psi_{k}|$, where $p_j$ and $p_k$ are the corresponding (eigenvalue) probability distributions.

If one of the states is a pure state ($\rho=|\psi_{\rho}\rangle \langle \psi_{\rho}|$), the above result boils down to the fidelity between a pure and a mixed state:

\begin{align}
F(\sigma, |\psi_{\rho}\rangle) = \langle\psi_{\rho}|\sigma|\psi_{\rho}\rangle = \sum_{j,k=0}^{d-1}\psi_{j}^{*}\sigma_{j,k}\psi_{k}.
\end{align}

In a noise-free environment, both the evolved state and the target states are pure states, the state fidelity thus becomes:

\begin{align}
F(\rho, \sigma) = |\langle\psi_{\rho}|\psi_{\sigma}\rangle|^2.
\end{align}

# &nbsp; <a href="#"><img valign="middle" height="45px" src="https://img.icons8.com/book" width="45" hspace="0px" vspace="0px"></a> References<a name="ref" />

\[1] Nielsen MA, Chuang IL. 2010. Quantum Computation and Quantum Information. New York: [Cambridge Univ. Press.](https://doi.org/10.1017/CBO9780511976667) 10th Anniv. Ed. 
- Corollary 4.2, pg. 176: Gate decomposition.
- Theorem 4.3, pg. 207: Trotter formula. 
- Chapter 4.7.2, pg. 206: The quantum simulation algorithm. 
- Chapter 9.2.2, pg. 409: State fidelity.
  
\[2] Griffiths, David J., and Darrell F. Schroeter. Introduction to quantum mechanics. Cambridge University Press, 2018.
- Section 3.2.3, pg. 103: scalar wave function.

\[3] [ibmq-qsim-challenge.ipynb](https://github.com/qiskit-community/open-science-prize-2021/blob/main/ibmq-qsim-challenge.ipynb).

\[4] [ibmq-qsim-sup-mat.ipynb](https://github.com/qiskit-community/open-science-prize-2021/blob/main/ibmq-qsim-sup-mat.ipynb).